## MANUAL CURATION: ANABAENA CYLINDRICA

### Metabolite annotation


Anotar nombres de los metabolitos con sus identificadores en las bases de datos KEGG y BIGG

In [ ]:
import cobra

# Cargar el modelo metabólico
model = cobra.io.read_sbml_model("C:/Users/elena/Documents/UNI/Asignaturas/TFG/Metabolic_models/anabaena_cylindrica.sbml")




Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00443_c0 "ABEE_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd02920_c0 "2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydropteridinediphosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00012_c0 "PPi_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00067_c0 "H_plus__c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00683_c0 "Dihydropteroate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00002_c0 "ATP_c0">
Use of the species charge attribute is discouraged, use fbc:charge

Descargar del github de modelSEED la base de datos que relaciona los nombres de los compuestos con sus respectivos identificadores en KEGG y BIGG

In [ ]:
###Descargar base de datos model seed para anotar compuestos
import requests

# URL del archivo Raw en GitHub
url = "https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/refs/heads/master/Biochemistry/compounds.tsv"

# Nombre con el que quieres guardar el archivo
file_name = "Compounds.tsv"

# Descargar el archivo
response = requests.get(url)
if response.status_code == 200:
    with open(file_name, "wb") as f:
        f.write(response.content)
    print(f"Archivo guardado como {file_name}")
else:
    print(f"Error al descargar el archivo: {response.status_code}")


Verificar si ya existen en el modelo algunos metabolitos anotados

In [2]:

# Verificar si hay anotaciones en los metabolitos
for met in model.metabolites:
    print(f"🔍 {met.id} ({met.name}) → Anotaciones: {met.annotation}")


🔍 cpd00443_c0 (ABEE_c0) → Anotaciones: {}
🔍 cpd02920_c0 (2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydropteridinediphosphate_c0) → Anotaciones: {}
🔍 cpd00012_c0 (PPi_c0) → Anotaciones: {}
🔍 cpd00067_c0 (H_plus__c0) → Anotaciones: {}
🔍 cpd00683_c0 (Dihydropteroate_c0) → Anotaciones: {}
🔍 cpd00002_c0 (ATP_c0) → Anotaciones: {}
🔍 cpd00033_c0 (Glycine_c0) → Anotaciones: {}
🔍 cpd00506_c0 (gamma_Glutamylcysteine_c0) → Anotaciones: {}
🔍 cpd00008_c0 (ADP_c0) → Anotaciones: {}
🔍 cpd00009_c0 (Phosphate_c0) → Anotaciones: {}
🔍 cpd00042_c0 (GSH_c0) → Anotaciones: {}
🔍 cpd00005_c0 (NADPH_c0) → Anotaciones: {}
🔍 cpd15560_c0 (Ubiquinone_8_c0) → Anotaciones: {}
🔍 cpd00006_c0 (NADP_c0) → Anotaciones: {}
🔍 cpd15561_c0 (Ubiquinol_8_c0) → Anotaciones: {}
🔍 cpd00001_c0 (H2O_c0) → Anotaciones: {}
🔍 cpd01759_c0 (N_Acyl_L_aspartate_c0) → Anotaciones: {}
🔍 cpd00041_c0 (L_Aspartate_c0) → Anotaciones: {}
🔍 cpd00049_c0 (Carboxylic_acid_c0) → Anotaciones: {}
🔍 cpd00017_c0 (S_Adenosyl_L_methionine_c0) → Anotaciones: 

Como se ve ningun metabolito está anotado. 

Código para parsear la base de datos de compuestos de modelSEED y anotar los metabolitos al modelo 

In [3]:
import pandas as pd
import cobra
import re
### Mapear metabolitos desde base de datos modelseed

def clean_modelseed_id(met_id):
    """Elimina sufijos de compartimentos en los metabolitos de ModelSEED (ej. '_c0', '_e0', '_p0', '_b')."""
    return re.sub(r'(_[cepib]\d*|_b)$', '', met_id)  # Remueve "_c0", "_e0", "_p0", "_b"

def extract_db_ids(aliases):
    """Extrae identificadores de varias bases de datos desde 'aliases'."""
    if pd.isna(aliases):  # Si el campo está vacío, retornar None
        return {}

    alias_list = aliases.split("|")  # Separar los alias por '|'
    db_mapping = {}

    for alias in alias_list:
        parts = alias.split(":")
        if len(parts) == 2:
            db_name, db_id = parts
            if "KEGG" in db_name:
                db_mapping["kegg.compound"] = db_id.split(";")[0]  # Tomamos solo el primer KEGG ID
            elif "BiGG" in db_name:
                db_mapping["bigg.metabolite"] = db_id.split(";")[0]
            elif "MetaCyc" in db_name:
                db_mapping["metacyc.compound"] = db_id.split(";")[0]
            elif "AraCyc" in db_name:
                db_mapping["aracyc.compound"] = db_id.split(";")[0]
            elif "ChEBI" in db_name:
                db_mapping["chebi.compound"] = db_id.split(";")[0]

    return db_mapping  # Retorna un diccionario con todas las anotaciones encontradas

# 🔹 1️⃣ Cargar la base de datos de ModelSEED desde el archivo TSV
compounds_db = pd.read_csv("Compounds.tsv", sep="\t")

# 🔹 2️⃣ Crear un diccionario de conversión ModelSEED → {KEGG, BiGG, MetaCyc, etc.}
modelseed_to_annotations = {}

for index, row in compounds_db.iterrows():
    db_ids = extract_db_ids(row["aliases"])  # Extraer todas las bases de datos
    if db_ids:
        modelseed_to_annotations[row["id"]] = db_ids  # Asociar ModelSEED ID con las anotaciones



# 🔹 4️⃣ Buscar identificadores y asignarlos en el modelo
for met in model.metabolites:
    modelseed_id = clean_modelseed_id(met.id)  # Eliminar sufijos de compartimento
    annotations = modelseed_to_annotations.get(modelseed_id, {})  # Buscar en el diccionario

    if annotations:
        if not met.annotation:
            met.annotation = {}  # Asegurar que met.annotation es un diccionario
        met.annotation.update(annotations)  # Agregar todas las anotaciones encontradas
        print(f"✅ Asignadas anotaciones a {met.id}: {annotations}")

# 🔹 5️⃣ Guardar el modelo con las anotaciones completas
cobra.io.write_sbml_model(model, "anabaena_model_full_annotations.sbml")

print("✅ Anotación de metabolitos con múltiples bases de datos completada y guardada correctamente.")


C:\Users\elena\AppData\Local\Temp\ipykernel_8456\3560900483.py:36: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  compounds_db = pd.read_csv("Compounds.tsv", sep="\t")


✅ Asignadas anotaciones a cpd00443_c0: {'aracyc.compound': ' P-AMINO-BENZOATE', 'bigg.metabolite': ' 4abz', 'kegg.compound': ' C00568', 'metacyc.compound': ' P-AMINO-BENZOATE'}
✅ Asignadas anotaciones a cpd02920_c0: {'aracyc.compound': ' DIHYDROPTERIN-CH2OH-PP', 'bigg.metabolite': ' 2ahhmd', 'kegg.compound': ' C04807', 'metacyc.compound': ' DIHYDROPTERIN-CH2OH-PP'}
✅ Asignadas anotaciones a cpd00012_c0: {'aracyc.compound': ' PPI', 'bigg.metabolite': ' ppi', 'kegg.compound': ' C00013', 'metacyc.compound': ' PPI'}
✅ Asignadas anotaciones a cpd00067_c0: {'aracyc.compound': ' PROTON', 'bigg.metabolite': ' h', 'kegg.compound': ' C00080', 'metacyc.compound': ' PROTON'}
✅ Asignadas anotaciones a cpd00683_c0: {'aracyc.compound': ' 7-8-DIHYDROPTEROATE', 'bigg.metabolite': ' dhpt', 'kegg.compound': ' C00921', 'metacyc.compound': ' 7-8-DIHYDROPTEROATE'}
✅ Asignadas anotaciones a cpd00002_c0: {'aracyc.compound': ' ATP', 'bigg.metabolite': ' atp', 'kegg.compound': ' C00002', 'metacyc.compound': ' A

In [4]:
model_Anno = cobra.io.read_sbml_model("anabaena_model_full_annotations.sbml")

Verificar que las anotaciones se han añadido correctamente

In [5]:
#Verificar si hay anotaciones en los metabolitos en el nuevo modelo 
for met in model_Anno.metabolites:
    print(f"🔍 {met.id} ({met.name}) → Anotaciones: {met.annotation}")


🔍 cpd00443_c0 (ABEE_c0) → Anotaciones: {'aracyc.compound': ' P-AMINO-BENZOATE', 'bigg.metabolite': ' 4abz', 'kegg.compound': ' C00568', 'metacyc.compound': ' P-AMINO-BENZOATE'}
🔍 cpd02920_c0 (2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydropteridinediphosphate_c0) → Anotaciones: {'aracyc.compound': ' DIHYDROPTERIN-CH2OH-PP', 'bigg.metabolite': ' 2ahhmd', 'kegg.compound': ' C04807', 'metacyc.compound': ' DIHYDROPTERIN-CH2OH-PP'}
🔍 cpd00012_c0 (PPi_c0) → Anotaciones: {'aracyc.compound': ' PPI', 'bigg.metabolite': ' ppi', 'kegg.compound': ' C00013', 'metacyc.compound': ' PPI'}
🔍 cpd00067_c0 (H_plus__c0) → Anotaciones: {'aracyc.compound': ' PROTON', 'bigg.metabolite': ' h', 'kegg.compound': ' C00080', 'metacyc.compound': ' PROTON'}
🔍 cpd00683_c0 (Dihydropteroate_c0) → Anotaciones: {'aracyc.compound': ' 7-8-DIHYDROPTEROATE', 'bigg.metabolite': ' dhpt', 'kegg.compound': ' C00921', 'metacyc.compound': ' 7-8-DIHYDROPTEROATE'}
🔍 cpd00002_c0 (ATP_c0) → Anotaciones: {'aracyc.compound': ' ATP', 'bigg

### Anotacion de reacciones

Descargar base de datos de modelSEED de reacciones y sus respectivas asociaciones con identificadores KEGG y BIGG

In [6]:
import requests

### Descargar base de datos de reacciones de modelSEED
# URL del archivo RAW en GitHub
url = "https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/refs/heads/master/Biochemistry/reactions.tsv"
file_name = "reactions.tsv"

# Descargar el archivo
response = requests.get(url)
if response.status_code == 200:
    with open(file_name, "wb") as f:
        f.write(response.content)
    print(f"✅ Archivo '{file_name}' descargado correctamente.")
else:
    print(f"❌ Error al descargar el archivo: {response.status_code}")


KeyboardInterrupt: 

Parsear la base de datos de reacciones y anotar sus identificadores KEGG y BIGG

In [7]:
import pandas as pd
import cobra
import re

#Anotar reacciones
def clean_modelseed_rxn_id(rxn_id):
    """Elimina sufijos de compartimentos en los IDs de reacciones de ModelSEED (ej. '_c0', '_e0', '_p0', '_b')."""
    return re.sub(r'(_[cepib]\d*|_b)$', '', rxn_id)  # Remueve "_c0", "_e0", "_p0", "_b"

def extract_db_ids(aliases, ec_numbers):
    """Extrae identificadores de varias bases de datos desde 'aliases' y EC numbers."""
    db_mapping = {}

    # Extraer información de aliases
    if pd.notna(aliases):  # Si el campo no está vacío
        alias_list = aliases.split("|")  # Separar los alias por '|'
        for alias in alias_list:
            parts = alias.split(":")
            if len(parts) == 2:
                db_name, db_id = parts
                if "KEGG" in db_name:
                    db_mapping["kegg.reaction"] = db_id.split(";")[0]  # Tomamos solo el primer KEGG ID
                elif "BiGG" in db_name:
                    db_mapping["bigg.reaction"] = db_id.split(";")[0]
                elif "MetaCyc" in db_name:
                    db_mapping["metacyc.reaction"] = db_id.split(";")[0]
                elif "AraCyc" in db_name:
                    db_mapping["aracyc.reaction"] = db_id.split(";")[0]

    # Extraer número EC (si está disponible)
    if pd.notna(ec_numbers) and ec_numbers != "null":
        db_mapping["ec_number"] = ec_numbers.split(";")[0]  # Tomamos solo el primer EC Number

    return db_mapping  # Retorna un diccionario con todas las anotaciones encontradas

# 🔹 1️⃣ Cargar la base de datos de ModelSEED desde el archivo TSV
reactions_db = pd.read_csv("reactions.tsv", sep="\t")

# 🔹 2️⃣ Crear un diccionario de conversión ModelSEED Reaction ID → {KEGG, BiGG, MetaCyc, etc.}
modelseed_to_annotations = {}

for index, row in reactions_db.iterrows():
    db_ids = extract_db_ids(row["aliases"], row["ec_numbers"])  # Extraer todas las bases de datos
    if db_ids:
        modelseed_to_annotations[row["id"]] = db_ids  # Asociar ModelSEED ID con las anotaciones

# 🔹 3️⃣ Cargar el modelo metabólico generado por ModelSEED
model = model_Anno

# 🔹 4️⃣ Buscar identificadores y asignarlos en el modelo
for rxn in model.reactions:
    modelseed_rxn_id = clean_modelseed_rxn_id(rxn.id)  # Eliminar sufijos de compartimento
    annotations = modelseed_to_annotations.get(modelseed_rxn_id, {})  # Buscar en el diccionario

    if annotations:
        if not rxn.annotation:
            rxn.annotation = {}  # Asegurar que rxn.annotation es un diccionario
        rxn.annotation.update(annotations)  # Agregar todas las anotaciones encontradas
        print(f"✅ Asignadas anotaciones a {rxn.id}: {annotations}")

# 🔹 5️⃣ Guardar el modelo con las anotaciones completas
cobra.io.write_sbml_model(model, "anabaena_model_rxn_full_annotations.sbml")

print("✅ Anotación de reacciones con múltiples bases de datos completada y guardada correctamente.")



✅ Asignadas anotaciones a rxn02201_c0: {'aracyc.reaction': ' H2PTEROATESYNTH-RXN', 'bigg.reaction': ' DHPS2', 'kegg.reaction': ' R03067', 'metacyc.reaction': ' H2PTEROATESYNTH-RXN', 'ec_number': '2.5.1.15'}
✅ Asignadas anotaciones a rxn00351_c0: {'aracyc.reaction': ' GLUTATHIONE-SYN-RXN', 'bigg.reaction': ' GSH2', 'kegg.reaction': ' R00497', 'metacyc.reaction': ' GLUTATHIONE-SYN-RXN', 'ec_number': '6.3.2.3'}
✅ Asignadas anotaciones a rxn08977_c0: {'aracyc.reaction': ' NQOR-RXN', 'bigg.reaction': ' NADPHQR2', 'metacyc.reaction': ' NQOR-RXN', 'ec_number': '1.6.5.2|1.6.99.6'}
✅ Asignadas anotaciones a rxn00390_c0: {'kegg.reaction': ' R00546', 'ec_number': '3.5.1.15'}
✅ Asignadas anotaciones a rxn08180_c0: {'bigg.reaction': ' BTS4', 'ec_number': '2.8.1.6'}
✅ Asignadas anotaciones a rxn11567_c0: {'kegg.reaction': ' R03355', 'ec_number': '3.2.1.23'}
✅ Asignadas anotaciones a rxn00423_c0: {'aracyc.reaction': ' SERINE-O-ACETTRAN-RXN', 'bigg.reaction': ' SERAT', 'kegg.reaction': ' R00586', 'met

In [8]:
import cobra
model_Anno2 = cobra.io.read_sbml_model("anabaena_model_rxn_full_annotations.sbml")


### Gene annotation

Verificar si hay genes anotados

In [9]:

# Verificar cuántos genes tienen anotaciones
for gene in model_Anno2.genes:
    print(f"{gene.id}: {gene.annotation}")


fig|2692879.3.peg.4805: {}
fig|2692879.3.peg.5268: {}
fig|2692879.3.peg.4584: {}
fig|2692879.3.peg.3168: {}
Unknown: {}
fig|2692879.3.peg.1786: {}
fig|2692879.3.peg.2329: {}
fig|2692879.3.peg.4394: {}
fig|2692879.3.peg.5672: {}
fig|2692879.3.peg.5965: {}
fig|2692879.3.peg.3610: {}
fig|2692879.3.peg.4539: {}
fig|2692879.3.peg.2179: {}
fig|2692879.3.peg.3243: {}
fig|2692879.3.peg.1159: {}
fig|2692879.3.peg.1893: {}
fig|2692879.3.peg.5246: {}
fig|2692879.3.peg.561: {}
fig|2692879.3.peg.3539: {}
fig|2692879.3.peg.2268: {}
fig|2692879.3.peg.6164: {}
fig|2692879.3.peg.1555: {}
fig|2692879.3.peg.3611: {}
fig|2692879.3.peg.4010: {}
fig|2692879.3.peg.1477: {}
fig|2692879.3.peg.1663: {}
fig|2692879.3.peg.3255: {}
fig|2692879.3.peg.3320: {}
fig|2692879.3.peg.1186: {}
fig|2692879.3.peg.5887: {}
fig|2692879.3.peg.6044: {}
fig|2692879.3.peg.4175: {}
fig|2692879.3.peg.1134: {}
fig|2692879.3.peg.6693: {}
fig|2692879.3.peg.3013: {}
fig|2692879.3.peg.4849: {}
fig|2692879.3.peg.6427: {}
fig|2692879.3.peg

In [10]:
model=model_Anno2

Extraer identificadores PATRIC de los genes

In [11]:


# 🔹 Extraer los IDs de genes
gene_ids = [gene.id for gene in model.genes]

# 🔹 Guardar en un archivo de texto
with open("gene_ids_patric.txt", "w") as f:
    for gene_id in gene_ids:
        f.write(gene_id + "\n")

print(f"✅ Se han guardado {len(gene_ids)} identificadores de genes en 'gene_ids_patric.txt'")



✅ Se han guardado 1406 identificadores de genes en 'gene_ids_patric.txt'


Limpiar identificadores PATRIC para poder mapearlos a los identificadores KEGG

In [12]:
# 🔹 Cargar y procesar el archivo
input_file = "refseq.txt"  # Cambia esto por el nombre de tu archivo
output_file = "clean_refseq.txt"

# 🔹 Leer el archivo, quitar ".1" y guardar el resultado
with open(input_file, "r") as f:
    lines = f.readlines()

# 🔹 Eliminar el sufijo ".1" de cada identificador
cleaned_lines = [line.strip().split(".")[0] + "\n" for line in lines]

# 🔹 Guardar el archivo limpio
with open(output_file, "w") as f:
    f.writelines(cleaned_lines)

print(f"✅ Archivo limpio guardado como: {output_file}")


✅ Archivo limpio guardado como: clean_refseq.txt


Buscar en la base de datos KEGG con su API los identificadores de los genes 

In [13]:
import pandas as pd
import cobra

# 🔹 1️⃣ Cargar el archivo de mapeo de IDs
mapping_file = "BVBRC_id_mapping (1).csv"  # Nombre del archivo CSV
mapping_df = pd.read_csv(mapping_file)

# 🔹 2️⃣ Crear un diccionario con el mapping PATRIC → RefSeq
patric_to_refseq = dict(zip(mapping_df["Source"], mapping_df["Target"]))

# 🔹 4️⃣ Asignar identificadores de RefSeq a los genes del modelo
for gene in model.genes:
    if gene.id in patric_to_refseq:
        gene.annotation["refseq"] = patric_to_refseq[gene.id]  # Asignar RefSeq

# 🔹 5️⃣ Guardar el modelo corregido
cobra.io.write_sbml_model(model, "model_genes.xml")

print("✅ Identificadores de RefSeq asignados correctamente y modelo guardado.")


✅ Identificadores de RefSeq asignados correctamente y modelo guardado.


### Asignacion de formulas a los metabolitos

Los metabolitos no tenian fórmula asignada lo que hacía imposible el balance de masa. La fórmula de llos metabolitos se extrajo de la propia base de datos de compuestos de modelSEED obtenida de github

In [14]:
import pandas as pd
import cobra

# 🔹 1️⃣ Cargar la base de datos de compuestos de ModelSEED, forzando las columnas a ser texto
compounds_db = pd.read_csv("Compounds.tsv", sep="\t", dtype=str, low_memory=False)

# 🔹 2️⃣ Reemplazar valores NaN en la columna "formula" con cadenas vacías
compounds_db["formula"] = compounds_db["formula"].fillna("").astype(str)

# 🔹 3️⃣ Crear un diccionario con los identificadores de metabolitos y sus fórmulas
modelseed_to_formula = dict(zip(compounds_db["id"], compounds_db["formula"]))


# 🔹 5️⃣ Asignar la fórmula química a los metabolitos sin fórmula
for met in model.metabolites:
    clean_met_id = met.id.split("_")[0]  # Limpiar sufijos de compartimentos (_c0, _e0, etc.)
    new_formula = modelseed_to_formula.get(clean_met_id)
    met.formula = new_formula

# 🔹 6️⃣ Guardar el modelo corregido
cobra.io.write_sbml_model(model, "model_v3")

print("✅ Fórmulas químicas asignadas correctamente y modelo guardado.")



✅ Fórmulas químicas asignadas correctamente y modelo guardado.


### Anotacion de ontología

Descaragr bases de datos de ontología de GithHub de modelSEED

In [1]:
import json
import cobra
import requests

# 🔹 1️⃣ Definir la URL base de los archivos en GitHub
github_base_url = "https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/refs/heads/master/Ontologies/"
# 🔹 2️⃣ Lista de archivos de ontología a descargar
ontology_files = {
    "KEGG_KO_ontologyDictionary.json": "KBaseOntology.OntologyDictionary.KEGG_KO_ontologyDictionary.json",
    "KEGG_RXN_ontologyDictionary.json": "KBaseOntology.OntologyDictionary.KEGG_RXN_ontologyDictionary.json",
    "ModelSEED_RXN_ontologyDictionary.json": "KBaseOntology.OntologyDictionary.ModelSEED_RXN_ontologyDictionary.json",
    "MetaCyc_RXN_ontologyDictionary.json": "KBaseOntology.OntologyDictionary.MetaCyc_RXN_ontologyDictionary.json"
}

# 🔹 3️⃣ Descargar y guardar los archivos JSON
def download_json(file_name, github_file_name):
    url = github_base_url + github_file_name
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(file_name, "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"✅ {file_name} descargado correctamente.")
    else:
        print(f"❌ Error al descargar {file_name}")

for file_name, github_file_name in ontology_files.items():
    download_json(file_name, github_file_name)



✅ KEGG_KO_ontologyDictionary.json descargado correctamente.
✅ KEGG_RXN_ontologyDictionary.json descargado correctamente.
✅ ModelSEED_RXN_ontologyDictionary.json descargado correctamente.
✅ MetaCyc_RXN_ontologyDictionary.json descargado correctamente.


Agregar anotaciones de ontología

In [2]:
# 🔹 4️⃣ Función para cargar JSON
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# 🔹 5️⃣ Cargar los diccionarios de ontología
kegg_ko_dict = load_json("KEGG_KO_ontologyDictionary.json")
kegg_rxn_dict = load_json("KEGG_RXN_ontologyDictionary.json")
modelseed_rxn_dict = load_json("ModelSEED_RXN_ontologyDictionary.json")
metacyc_rxn_dict = load_json("MetaCyc_RXN_ontologyDictionary.json")

# 🔹 6️⃣ Cargar el modelo metabólico
model = cobra.io.read_sbml_model("model_v3")

# 🔹 7️⃣ Anotar términos GO en reacciones
for rxn in model.reactions:
    if rxn.id in modelseed_rxn_dict:
        rxn.annotation["GO"] = modelseed_rxn_dict[rxn.id]  # Asignar GO desde ModelSEED
    elif rxn.id in kegg_rxn_dict:
        rxn.annotation["GO"] = kegg_rxn_dict[rxn.id]  # Asignar GO desde KEGG
    elif rxn.id in metacyc_rxn_dict:
        rxn.annotation["GO"] = metacyc_rxn_dict[rxn.id]  # Asignar GO desde MetaCyc

# 🔹 8️⃣ Guardar el modelo con anotaciones mejoradas
cobra.io.write_sbml_model(model, "model_v4")

print("✅ Anotaciones de términos GO agregadas correctamente.")


✅ Anotaciones de términos GO agregadas correctamente.


In [3]:
import cobra

# 🔹 Cargar el modelo metabólico
model = cobra.io.read_sbml_model("model_v4")

# 🔹 1️⃣ Asignar SBO terms a los metabolitos
for met in model.metabolites:
    met.annotation["sbo"] = "SBO:0000247"  # Metabolito general

# 🔹 2️⃣ Asignar SBO terms a las reacciones
for rxn in model.reactions:
    if "exchange" in rxn.id.lower() or rxn.id.startswith("EX_"):
        rxn.annotation["sbo"] = "SBO:0000627"  # Reacción de intercambio
    elif "transport" in rxn.id.lower():
        rxn.annotation["sbo"] = "SBO:0000185"  # Reacción de transporte
    elif "biomass" in rxn.id.lower() or "bio" in rxn.id.lower():
        rxn.annotation["sbo"] = "SBO:0000629"  # Reacción de biomasa
    else:
        rxn.annotation["sbo"] = "SBO:0000176"  # Reacción metabólica

# 🔹 3️⃣ Asignar SBO terms a los genes
for gene in model.genes:
    gene.annotation["sbo"] = "SBO:0000243"  # Gen general

# 🔹 Guardar el modelo corregido
cobra.io.write_sbml_model(model, "model_v5")

print("✅ Términos SBO asignados correctamente y modelo guardado.")


✅ Términos SBO asignados correctamente y modelo guardado.


### Reaction constraints

El modelo se construyó sobre un medio completo, por lo que las reacciones no suelen tener restriccion, añadimos un limite arbitrario para restringir la entrada y salida d enutrientes

In [4]:
# Identificar reacciones sin restricciones
reactions_without_constraints = [rxn for rxn in model.reactions if rxn.lower_bound == -1000 and rxn.upper_bound == 1000]

print(f"Se han identificado {len(reactions_without_constraints)} reacciones sin restricciones.")

Se han identificado 776 reacciones sin restricciones.


In [5]:
# Establecer límites más realistas para reacciones metabólicas
for rxn in model.reactions:
    if rxn.lower_bound == -1000 and rxn.upper_bound == 1000:
        if "EX_" in rxn.id:  # Para reacciones de intercambio
            rxn.lower_bound = -1  # Limitar la entrada de metabolitos
            rxn.upper_bound = 1  # Permitir salida si es posible
        elif "DM_" in rxn.id:  # Para reacciones de demanda
            rxn.lower_bound = 0
            rxn.upper_bound = 1  # Controlar la producción
        else:  # Para reacciones metabólicas internas
            rxn.lower_bound = -1  # Limitar la reversibilidad
            rxn.upper_bound = 1

print("✅ Restricciones de flujo aplicadas correctamente.")


✅ Restricciones de flujo aplicadas correctamente.


In [6]:
cobra.io.write_sbml_model(model, "modelo_vsiete.xml")
print("✅ Modelo guardado correctamente.")


✅ Modelo guardado correctamente.


In [7]:
model=cobra.io.read_sbml_model("modelo_vsiete.xml")

In [8]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_b,EX_cpd00007_b,0.2141,0,0.00%
cpd00023_b,EX_cpd00023_b,0.4347,5,45.23%
cpd00028_b,EX_cpd00028_b,0.0005986,34,0.42%
cpd00030_b,EX_cpd00030_b,0.0002993,0,0.00%
cpd00034_b,EX_cpd00034_b,0.0002993,0,0.00%
cpd00038_b,EX_cpd00038_b,0.01309,10,2.72%
cpd00045_b,EX_cpd00045_b,0.01245,10,2.59%
cpd00048_b,EX_cpd00048_b,0.0002993,0,0.00%
cpd00058_b,EX_cpd00058_b,0.0002993,0,0.00%
cpd00063_b,EX_cpd00063_b,0.0002993,0,0.00%


### Biomass inconsistencies

Resolver incosnistencias en la ecuacion de biomasa

Primero, hay metabolitos e la reacción que no tienen fórmula, estos se corresponden con términos abstractos, los eliminamos

In [9]:
import cobra

# Cargar el modelo
model = cobra.io.read_sbml_model("modelo_vsiete.xml")

# Buscar la reacción de biomasa por su ID "bio1"
biomass_rxn = model.reactions.get_by_id("bio1")

# Verificar qué metabolitos en la biomasa no tienen fórmula
metabolites_missing_formula = [met for met in biomass_rxn.metabolites if not met.formula]

# Mostrar metabolitos sin fórmula
print("⚠️ Metabolitos en la reacción de biomasa (bio1) sin fórmula química:")
for met in metabolites_missing_formula:
    print(f"{met.id}: {met.name}")



⚠️ Metabolitos en la reacción de biomasa (bio1) sin fórmula química:
cpd17041_c0: Protein_biosynthesis_c0
cpd17043_c0: RNA_transcription_c0
cpd17042_c0: DNA_replication_c0
cpd11416_c0: Biomass_c0


In [10]:


# Buscar la reacción de biomasa
biomass_rxn = model.reactions.get_by_id("bio1")

# Lista de metabolitos abstractos a eliminar
abstract_metabolites = ["cpd17042_c0", "cpd17043_c0", "cpd17041_c0", "cpd11416_c0"]

# Eliminar los metabolitos de la ecuación de biomasa
for met_id in abstract_metabolites:
    if met_id in biomass_rxn.metabolites:
        biomass_rxn.subtract_metabolites({model.metabolites.get_by_id(met_id): biomass_rxn.get_coefficient(met_id)})
        print(f"❌ Eliminado {met_id} de la reacción de biomasa.")

# Guardar el modelo corregido
cobra.io.write_sbml_model(model, "modelo_v8")
print("✅ Modelo guardado sin metabolitos abstractos en biomasa.")


✅ Modelo guardado sin metabolitos abstractos en biomasa.


In [11]:


# Buscar la reacción de biomasa
biomass_rxn = model.reactions.get_by_id("bio1")

# Imprimir la ecuación de la reacción de biomasa con nombres de metabolitos
print(f"🔬 Ecuación de la reacción de biomasa ({biomass_rxn.id}):")
print(biomass_rxn.build_reaction_string(use_metabolite_names=True))
# Mostrar metabolitos individuales con coeficientes y fórmulas químicas
print("\n📜 Lista detallada de metabolitos en la reacción de biomasa:")
print(f"{'Coeficiente':<15} {'ID Metabolito':<20} {'Nombre':<40} {'Fórmula química'}")
print("="*100)

for met, coeff in biomass_rxn.metabolites.items():
    formula = met.formula if met.formula else "❌ No tiene fórmula"
    print(f"{coeff:<15} {met.id:<20} {met.name:<40} {formula}")


🔬 Ecuación de la reacción de biomasa (bio1):
35.5403092430435 H2O_c0 + 40.1101757365074 ATP_c0 + 0.00309646685192537 NAD_c0 + 0.00309646685192537 NADP_c0 + 0.00309646685192537 CoA_c0 + 0.00309646685192537 FAD_c0 + 0.00309646685192537 Pyridoxal_phosphate_c0 + 0.00309646685192537 S_Adenosyl_L_methionine_c0 + 0.219088153012743 L_Glutamate_c0 + 0.00309646685192537 Heme_c0 + 0.00309646685192537 Mn2_plus__c0 + 0.509869786991038 Glycine_c0 + 0.00309646685192537 Zn2_plus__c0 + 0.427934380173264 L_Alanine_c0 + 0.135406821203723 GTP_c0 + 0.285438020490179 L_Lysine_c0 + 0.200830806928348 L_Aspartate_c0 + 0.00309646685192537 GSH_c0 + 0.00309646685192537 Sulfate_c0 + 0.246696822701341 L_Arginine_c0 + 0.0841036156544863 CTP_c0 + 0.219088153012743 L_Glutamine_c0 + 0.179456352975885 L_Serine_c0 + 0.00309646685192537 TPP_c0 + 0.00309646685192537 Cu2_plus__c0 + 0.127801422590767 L_Methionine_c0 + 0.0908319049068452 UTP_c0 + 0.00309646685192537 Ca2_plus__c0 + 0.0472019191450218 L_Tryptophan_c0 + 0.154519

In [12]:
import cobra

# Cargar el modelo metabólico desde un archivo SBML
#model = cobra.io.read_sbml_model("modelo_v8.xml")

# Buscar la reacción de biomasa
biomass_rxn = model.reactions.get_by_id("bio1")

# Lista de metabolitos a eliminar (según los identificados previamente)
metabolitos_a_eliminar = {"cpd17042_c0", "cpd17043_c0", "cpd17041_c0", "cpd11416_c0"}

# Filtrar los metabolitos que están en la reacción de biomasa
metabolitos_presentes = [met for met in biomass_rxn.metabolites if met.id in metabolitos_a_eliminar]

# Eliminar los metabolitos de la reacción de biomasa
if metabolitos_presentes:
    biomass_rxn.subtract_metabolites({met: biomass_rxn.metabolites[met] for met in metabolitos_presentes})
    print(f"✅ Eliminados: {[met.id for met in metabolitos_presentes]}")
else:
    print("⚠️ Ningún metabolito de la lista se encontró en la reacción de biomasa.")


✅ Eliminados: ['cpd17041_c0', 'cpd17043_c0', 'cpd17042_c0', 'cpd11416_c0']


Algunas fórmulas tienen grupos variables R que dificultan el balace estequiométrico de la reacción, los eliminamos de la fórmula reemplazandolos por protones

In [13]:
for met in biomass_rxn.metabolites:
    if met.formula is None:
        print(f"⚠️ {met.id} no tiene fórmula definida")
    elif "R" in met.formula:
        print(f"⚠️ {met.id} tiene una fórmula con un grupo R: {met.formula}")


⚠️ cpd11493_c0 tiene una fórmula con un grupo R: C11H22N2O7PRS
⚠️ cpd15665_c0 tiene una fórmula con un grupo R: C80H125N16O42R
⚠️ cpd12370_c0 tiene una fórmula con un grupo R: HOR
⚠️ cpd15666_c0 tiene una fórmula con un grupo R: C40H63N8O21R


In [14]:
for met in biomass_rxn.metabolites:
    if met.formula and "R" in met.formula:
        met.formula = met.formula.replace("R", "H")  # Sustituye "R" por Hidrógeno
        print(f"🔄 Se ha corregido la fórmula de {met.id}: {met.formula}")


🔄 Se ha corregido la fórmula de cpd11493_c0: C11H22N2O7PHS
🔄 Se ha corregido la fórmula de cpd15665_c0: C80H125N16O42H
🔄 Se ha corregido la fórmula de cpd12370_c0: HOH
🔄 Se ha corregido la fórmula de cpd15666_c0: C40H63N8O21H


In [15]:
total_mass = sum(met.formula_weight * abs(coef) for met, coef in biomass_rxn.metabolites.items() if met.formula)
print(f"⚖️ Masa total de la biomasa: {total_mass} g/mmol")



⚖️ Masa total de la biomasa: 42920.16751376454 g/mmol


In [16]:

# Guardar el modelo corregido
cobra.io.write_sbml_model(model, "modelo_v8.xml")

#### Ajustar coeficientes de biomasa

Ajustamos coeficientes de biomasa para escalarlo a 1

In [17]:
### Añadir metabolitos esenciales faltantes

# 📥 Cargar el modelo metabólico
#model = cobra.io.read_sbml_model("modelo_v8.xml")  # Asegúrate de usar el modelo correcto

# 🔍 Obtener la reacción de biomasa
biomass_rxn = model.reactions.get_by_id("bio1")

# 📌 Definir el coeficiente teórico basado en literatura de bacterias similares
fmn_coefficient = -0.005  # Ajustable según datos experimentales

# 🔄 **Añadir el metabolito `cpd00050_c0` a la biomasa**
if "cpd00050_c0" in model.metabolites:
    fmn_met = model.metabolites.get_by_id("cpd00050_c0")
    biomass_rxn.add_metabolites({fmn_met: fmn_coefficient})
    print(f"✅ {fmn_met.id} añadido a la biomasa con coeficiente {fmn_coefficient:.6f}")
else:
    print("⚠️ Error: `cpd00050_c0` no se encuentra en el modelo. Verifica su presencia.")

# 💾 **Guardar el modelo corregido**
cobra.io.write_sbml_model(model, "modelo_vnueve.xml")
print("\n✅ Modelo guardado como 'modelo_fmn.xml' con FMN en la ecuación de biomasa.")


✅ cpd00050_c0 añadido a la biomasa con coeficiente -0.005000

✅ Modelo guardado como 'modelo_fmn.xml' con FMN en la ecuación de biomasa.


In [18]:
# Corregir coeficientes para consistencia biomasa. 
# 📥 Cargar el modelo
#model = cobra.io.read_sbml_model("modelo_vnueve.xml")

# 🔍 Obtener la reacción de biomasa
biomass_rxn = model.reactions.get_by_id("bio1")

# 📏 Calcular el factor de escalado
scaling_factor = 1.0 /  0.9228509154955418
print(f"🔢 Factor de ajuste: {scaling_factor:.6f}")

# 🔄 Aplicar el ajuste
new_metabolites = {met: coef * scaling_factor for met, coef in biomass_rxn.metabolites.items()}
biomass_rxn.add_metabolites(new_metabolites, combine=False)

# 💾 Guardar el modelo corregido
cobra.io.write_sbml_model(model, "modelo_v10.xml")
print("✅ Biomasa ajustada y modelo guardado.")


🔢 Factor de ajuste: 1.083599
✅ Biomasa ajustada y modelo guardado.


In [19]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_b,EX_cpd00007_b,0.2143,0,0.00%
cpd00023_b,EX_cpd00023_b,0.4347,5,45.15%
cpd00028_b,EX_cpd00028_b,0.0005986,34,0.42%
cpd00030_b,EX_cpd00030_b,0.0002993,0,0.00%
cpd00034_b,EX_cpd00034_b,0.0002993,0,0.00%
cpd00038_b,EX_cpd00038_b,0.01309,10,2.72%
cpd00045_b,EX_cpd00045_b,0.01244,10,2.59%
cpd00048_b,EX_cpd00048_b,0.0002993,0,0.00%
cpd00058_b,EX_cpd00058_b,0.0002993,0,0.00%
cpd00063_b,EX_cpd00063_b,0.0002993,0,0.00%


### Fluxes

El análisis de los flujos de cada reacción requiere una comparación con condiciones experimentales en los que se marcan los compuestos y se analizan llos flujos a través del metabolismo, como no podemos hacer esto se aplican restricciones generales

Identificamos reacciones esenciales para el crecimiento

In [20]:

from cobra.flux_analysis import single_reaction_deletion

# 📥 Cargar el modelo
#model = cobra.io.read_sbml_model("modelo_v10.xml")

# 🔍 Identificar reacciones esenciales usando eliminación de reacciones individuales
essential_reactions = single_reaction_deletion(model)

# 📌 Filtrar solo las reacciones cuya eliminación causa crecimiento nulo (esenciales)
essential_reactions = essential_reactions[essential_reactions["growth"] < 1e-6]

# 📜 Mostrar los resultados
print(f"🔍 Se encontraron {len(essential_reactions)} reacciones esenciales para el crecimiento:")
for rxn in essential_reactions.index[:10]:  # Mostrar solo las primeras 10
    print(f"✅ {rxn}")

# 💾 Guardar la lista en un archivo
essential_reactions.to_csv("reacciones_esenciales.csv", index=True)
print("\n✅ Lista de reacciones esenciales guardada en 'reacciones_esenciales.csv'.")




🔍 Se encontraron 382 reacciones esenciales para el crecimiento:
✅ 2
✅ 3
✅ 4
✅ 7
✅ 10
✅ 20
✅ 22
✅ 25
✅ 29
✅ 30

✅ Lista de reacciones esenciales guardada en 'reacciones_esenciales.csv'.


In [21]:
### Restricciones de flujo 
unbounded_rxns = [rxn for rxn in model.reactions if rxn.bounds == (-1000.0, 1000.0)]
print(f"🔍 {len(unbounded_rxns)} reacciones tienen flujo sin restricciones.")
for rxn in unbounded_rxns:
    print(f"- {rxn.id}: {rxn.reaction}")


🔍 0 reacciones tienen flujo sin restricciones.


Primero eliminamos las reacciones que vienen bloqueadas por defecto 

In [46]:
### Reacciones bloqueadas
from cobra.flux_analysis import find_blocked_reactions

# 🔍 Identificar reacciones bloqueadas
blocked_rxns = find_blocked_reactions(model)

print(f"🔍 Se encontraron {len(blocked_rxns)} reacciones bloqueadas.")

# 📜 Mostrar algunas reacciones bloqueadas
for rxn in blocked_rxns[:10]:
    print(f"🚫 Reacción bloqueada: {rxn}")

# 💾 Guardar la lista en un archivo
with open("blocked_reactions.txt", "w") as file:
    for rxn in blocked_rxns:
        file.write(f"{rxn}\n")

print("\n✅ Lista de reacciones bloqueadas guardada en 'blocked_reactions.txt'")


🔍 Se encontraron 580 reacciones bloqueadas.
🚫 Reacción bloqueada: EX_cpd15302_b
🚫 Reacción bloqueada: EX_cpd11416_b
🚫 Reacción bloqueada: EX_cpd04097_b
🚫 Reacción bloqueada: EX_cpd01012_b
🚫 Reacción bloqueada: EX_cpd00635_b
🚫 Reacción bloqueada: EX_cpd00971_b
🚫 Reacción bloqueada: EX_cpd00531_b
🚫 Reacción bloqueada: EX_cpd00104_b
🚫 Reacción bloqueada: rxn00351_c0
🚫 Reacción bloqueada: rxn00390_c0

✅ Lista de reacciones bloqueadas guardada en 'blocked_reactions.txt'


In [23]:


# 📂 Cargar la lista de reacciones bloqueadas desde el archivo
blocked_reactions_file = "blocked_reactions.txt"

with open(blocked_reactions_file, "r") as file:
    blocked_reactions = [line.strip() for line in file.readlines()]

print(f"🔍 Se encontraron {len(blocked_reactions)} reacciones bloqueadas en el archivo.")

# 🚧 **Desbloquear las reacciones bloqueadas**
for rxn_id in blocked_reactions:
    if rxn_id in model.reactions:
        rxn = model.reactions.get_by_id(rxn_id)
        
        # Permitir flujo en ambas direcciones con un rango moderado
        rxn.lower_bound = -1
        rxn.upper_bound = 1
        
        print(f"✅ Desbloqueada reacción: {rxn_id} ({rxn.reaction})")
    else:
        print(f"⚠️ Advertencia: {rxn_id} no está en el modelo.")

# 💾 **Guardar el modelo corregido**
#cobra.io.write_sbml_model(model, "modelo_desbloqueado.xml")
print("\n✅ Modelo corregido guardado como 'modelo_desbloqueado.xml'.")


🔍 Se encontraron 727 reacciones bloqueadas en el archivo.
⚠️ Advertencia: EX_cpd02701_b no está en el modelo.
✅ Desbloqueada reacción: EX_cpd11416_b (cpd11416_b <=> )
⚠️ Advertencia: EX_cpd11576_b no está en el modelo.
✅ Desbloqueada reacción: EX_cpd04097_b (cpd04097_b <=> )
⚠️ Advertencia: EX_cpd01030_b no está en el modelo.
✅ Desbloqueada reacción: EX_cpd01012_b (cpd01012_b <=> )
✅ Desbloqueada reacción: EX_cpd00635_b (cpd00635_b <=> )
✅ Desbloqueada reacción: EX_cpd00971_b (cpd00971_b <=> )
⚠️ Advertencia: EX_cpd00184_b no está en el modelo.
⚠️ Advertencia: EX_cpd00314_b no está en el modelo.
⚠️ Advertencia: EX_cpd00076_b no está en el modelo.
⚠️ Advertencia: EX_cpd01171_b no está en el modelo.
⚠️ Advertencia: EX_cpd03724_b no está en el modelo.
⚠️ Advertencia: EX_cpd00637_b no está en el modelo.
⚠️ Advertencia: EX_cpd08306_b no está en el modelo.
⚠️ Advertencia: EX_cpd03696_b no está en el modelo.
⚠️ Advertencia: EX_cpd01092_b no está en el modelo.
⚠️ Advertencia: EX_cpd01262_b no 

In [24]:
cobra.io.write_sbml_model(model, "modelo_v11.xml")

Restringimos arbitrariamente reacciones relacionadas con la producción de biomasa para evitar crecimiento excesivo 

In [25]:

# 📥 Cargar el modelo metabólico
model = cobra.io.read_sbml_model("modelo_v11.xml")

# 📜 Lista completa de reacciones a restringir
reacciones_a_restringir = [
    "EX_cpd00067_b", "EX_cpd00106_b", "EX_cpd00012_b", "EX_cpd00058_b", "EX_cpd00034_b",
    "EX_cpd00132_b", "EX_cpd00254_b", "EX_cpd00205_b", "EX_cpd10516_b", "EX_cpd03847_b",
    "EX_cpd00149_b", "EX_cpd00099_b", "EX_cpd00027_b", "EX_cpd00268_b", "EX_cpd01080_b",
    "EX_cpd00039_b", "EX_cpd00241_b", "EX_cpd00793_b", "EX_cpd00030_b", "EX_cpd00264_b",
    "EX_cpd00111_b", "EX_cpd00038_b", "EX_cpd00028_b", "EX_cpd00063_b", "EX_cpd15606_b",
    "EX_cpd00322_b", "EX_cpd11606_b", "EX_cpd00115_b", "EX_cpd00218_b", "rxn00423_c0",
    "rxn00364_c0", "rxn05561_c0", "rxn03408_c0", "rxn05440_c0", "rxn00973_c0",
    "rxn02000_c0", "rxn05457_c0", "rxn05392_c0", "rxn12008_c0", "rxn00199_c0",
    "rxn09240_c0", "rxn00519_c0", "rxn00258_c0", "rxn00800_c0", "rxn00179_c0",
    "rxn05367_c0", "rxn05654_c0", "rxn02320_c0", "rxn02504_c0", "rxn00262_c0",
    "rxn05404_c0", "rxn01486_c0", "rxn00898_c0", "rxn05444_c0", "rxn05040_c0",
    "rxn05400_c0", "rxn02988_c0", "rxn00678_c0", "rxn11268_c0", "rxn00710_c0",
    "rxn00470_c0", "rxn00910_c0", "rxn03181_c0", "rxn02331_c0", "rxn06493_c0",
    "rxn12846_c0", "rxn00337_c0", "rxn05434_c0", "rxn05733_c0", "rxn12512_c0",
    "rxn00902_c0", "rxn01268_c0", "rxn00225_c0", "rxn00148_c0", "rxn01102_c0",
    "rxn05358_c0", "rxn00100_c0", "rxn00789_c0", "rxn02374_c0", "rxn05437_c0",
    "rxn00781_c0", "rxn05319_c0", "rxn09202_c0", "rxn00097_c0", "rxn02212_c0",
    "rxn00695_c0", "rxn10481_c0", "rxn00692_c0", "rxn00802_c0", "rxn02160_c0",
    "rxn01997_c0", "rxn03958_c0", "rxn00178_c0", "rxn01362_c0", "rxn10060_c0",
    "rxn02933_c0", "rxn00727_c0", "rxn02268_c0", "rxn00612_c0", "rxn02159_c0",
    "rxn01111_c0", "rxn00300_c0", "rxn05938_c0", "rxn06078_c0", "rxn05435_c0",
    "rxn11946_c0", "rxn01504_c0", "rxn00907_c0", "rxn05458_c0", "rxn02404_c0",
    "rxn12847_c0", "rxn05443_c0", "rxn00336_c0", "rxn03891_c0", "rxn01465_c0",
    "rxn01643_c0", "rxn00392_c0", "rxn00747_c0", "rxn10113_c0", "rxn02175_c0",
    "rxn02835_c0", "rxn01675_c0", "rxn03536_c0", "rxn05150_c0", "rxn05359_c0",
    "rxn00555_c0", "rxn03080_c0", "rxn00849_c0", "rxn00213_c0", "rxn01740_c0",
    "rxn00251_c0", "rxn03910_c0", "rxn00237_c0", "rxn08311_c0", "rxn03175_c0",
    "rxn10227_c0", "rxn09210_c0", "rxn00285_c0", "rxn05371_c0", "rxn08615_c0",
    "rxn00126_c0", "rxn08618_c0", "rxn00058_c0", "rxn00299_c0", "rxn00904_c0",
    "rxn03159_c0", "rxn01000_c0", "rxn01300_c0", "rxn03908_c0", "rxn01100_c0",
    "rxn00838_c0", "rxn00512_c0", "rxn05396_c0", "rxn01255_c0", "rxn01211_c0",
    "rxn00527_c0", "rxn03537_c0", "rxn00966_c0", "rxn00173_c0", "rxn08528_c0",
    "rxn01669_c0", "rxn10122_c0", "rxn08352_c0", "rxn09106_c0", "rxn05508_c0",
    "rxn03135_c0", "rxn10265_c0", "rxn00915_c0", "rxn01116_c0", "rxn02811_c0",
    "rxn03062_c0", "rxn00868_c0", "rxn00670_c0", "rxn01544_c0", "rxn00863_c0"
]

# 🔬 **Definir restricciones genéricas**
for rxn_id in reacciones_a_restringir:
    if rxn_id in model.reactions:
        rxn = model.reactions.get_by_id(rxn_id)
        
        if "EX_" in rxn_id:  # 🚀 Reacciones de intercambio
            rxn.lower_bound = -1  # Permitir absorción
            rxn.upper_bound = 1  # Permitir secreción
        elif "rxn" in rxn_id:  # 🔬 Reacciones metabólicas internas
            rxn.lower_bound = -1
            rxn.upper_bound = 1
        else:  # 🛠️ Otras reacciones (ajuste moderado)
            rxn.lower_bound = -1
            rxn.upper_bound = 1
        
        print(f"✅ Restricción aplicada: {rxn_id} → [{rxn.lower_bound}, {rxn.upper_bound}]")
    else:
        print(f"⚠️ Advertencia: {rxn_id} no encontrado en el modelo.")

# 💾 **Guardar el modelo corregido**
cobra.io.write_sbml_model(model, "modelo_v12.xml")

print("\n✅ Modelo corregido guardado como 'modelo_restringido.xml'.")


✅ Restricción aplicada: EX_cpd00067_b → [-1, 1]
⚠️ Advertencia: EX_cpd00106_b no encontrado en el modelo.
✅ Restricción aplicada: EX_cpd00012_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00058_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00034_b → [-1, 1]
⚠️ Advertencia: EX_cpd00132_b no encontrado en el modelo.
✅ Restricción aplicada: EX_cpd00254_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00205_b → [-1, 1]
✅ Restricción aplicada: EX_cpd10516_b → [-1, 1]
✅ Restricción aplicada: EX_cpd03847_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00149_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00099_b → [-1, 1]
⚠️ Advertencia: EX_cpd00027_b no encontrado en el modelo.
✅ Restricción aplicada: EX_cpd00268_b → [-1, 1]
✅ Restricción aplicada: EX_cpd01080_b → [-1, 1]
⚠️ Advertencia: EX_cpd00039_b no encontrado en el modelo.
✅ Restricción aplicada: EX_cpd00241_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00793_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00030_b → [-1, 1]
✅ Restricción aplicada: EX_cpd00264_b → [-1, 1]


### Precursores de biomasa bloquedos 

El modelo no crece porque algunos precursores de biomasa están bloquados, hay que desbloquearlos

In [26]:
import cobra

# Cargar el modelo
#model = cobra.io.read_sbml_model("modelo_v12.xml")

# Lista de precursores bloqueados
blocked_precursors =  ["cpd15665_c0","cpd11493_c0","cpd00166_c0"]

# Verificar si los metabolitos tienen rutas funcionales
for met_id in blocked_precursors:
    met = model.metabolites.get_by_id(met_id)
    related_reactions = [rxn.id for rxn in met.reactions]
    
    
    print(f"\n🔍 Metabolito: {met_id}")
    print(f"📌 Reacciones asociadas: {related_reactions if related_reactions else '❌ Ninguna'}")



🔍 Metabolito: cpd15665_c0
📌 Reacciones asociadas: ['bio1', 'rxn10199_c0']

🔍 Metabolito: cpd11493_c0
📌 Reacciones asociadas: ['rxn05388_c0', 'rxn05455_c0', 'rxn05363_c0', 'rxn05384_c0', 'rxn05459_c0', 'rxn05404_c0', 'rxn05425_c0', 'rxn05359_c0', 'rxn05460_c0', 'rxn05454_c0', 'rxn05392_c0', 'rxn05358_c0', 'rxn05465_c0', 'rxn06723_c0', 'rxn05400_c0', 'rxn05417_c0', 'rxn08433_c0', 'rxn05452_c0', 'rxn08438_c0', 'rxn05451_c0', 'rxn05345_c0', 'rxn05371_c0', 'rxn05375_c0', 'rxn05346_c0', 'rxn05413_c0', 'rxn05409_c0', 'rxn05379_c0', 'rxn09436_c0', 'bio1', 'rxn05456_c0', 'rxn05343_c0', 'rxn06848_c0', 'rxn05344_c0', 'rxn05408_c0', 'rxn05349_c0', 'rxn06865_c0', 'rxn05458_c0', 'rxn06729_c0', 'rxn05367_c0', 'rxn05396_c0', 'rxn05421_c0', 'rxn05457_c0', 'rxn05350_c0', 'rxn05383_c0', 'rxn05429_c0', 'rxn06023_c0', 'rxn05453_c0', 'rxn05348_c0', 'rxn05347_c0']

🔍 Metabolito: cpd00166_c0
📌 Reacciones asociadas: ['bio1', 'rxn03538_c0']


Para estos precursores no existe una reaccion de intercambio del medio, la añadimos

In [27]:
for met_id in blocked_precursors:
    exchange_met_id = met_id.replace("_c0", "_b")
    exchange_rxn = f"EX_{exchange_met_id}"
    if exchange_rxn in model.reactions:
        model.reactions.get_by_id(exchange_rxn).lower_bound = -10  # Permitir su consumo
        print(f"✅ Se ajustó la reacción de intercambio para {exchange_met_id}")
    else:
        print(f"⚠️ No existe una reacción de intercambio para {exchange_met_id}")


⚠️ No existe una reacción de intercambio para cpd15665_b
⚠️ No existe una reacción de intercambio para cpd11493_b
⚠️ No existe una reacción de intercambio para cpd00166_b


In [53]:
#rer recciones de intercmbio r os metboitos boquedos 
for met_id in blocked_precursors:
    exchange_met_id = met_id.replace("_c0", "_b")
    exchange_rxn = f"EX_{exchange_met_id}"
    if exchange_rxn not in model.reactions:
        rxn = cobra.Reaction(exchange_rxn)
        rxn.add_metabolites({model.metabolites.get_by_id(met_id): -1.0})
        rxn.lower_bound = -1  # Permitir su consumo
        rxn.upper_bound = 1  # Mantener el flujo abierto
        model.add_reactions([rxn])
        print(f"✅ Se agregó la reacción de intercambio: {exchange_rxn}")


✅ Se agregó la reacción de intercambio: EX_cpd15665_b
✅ Se agregó la reacción de intercambio: EX_cpd11493_b
✅ Se agregó la reacción de intercambio: EX_cpd00166_b


In [29]:
# Ver si hy recciones boquedas que afecten al metabolito. 
from cobra.flux_analysis import find_blocked_reactions

# Obtener reacciones bloqueadas
blocked_reactions = find_blocked_reactions(model)

# Revisar si los precursores están involucrados en ellas
for met_id in blocked_precursors:
    met = model.metabolites.get_by_id(met_id)
    related_blocked_reactions = [rxn.id for rxn in met.reactions if rxn.id in blocked_reactions]
    
    if related_blocked_reactions:
        print(f"⚠️ {met_id} está en reacciones bloqueadas: {related_blocked_reactions}")
    else:
        print(f"✅ {met_id} no está en reacciones bloqueadas")


✅ cpd15665_c0 no está en reacciones bloqueadas
⚠️ cpd11493_c0 está en reacciones bloqueadas: ['rxn05455_c0', 'rxn05459_c0', 'rxn05425_c0', 'rxn05460_c0', 'rxn05417_c0', 'rxn08433_c0', 'rxn08438_c0', 'rxn05451_c0', 'rxn05345_c0', 'rxn05346_c0', 'rxn05413_c0', 'rxn05409_c0', 'rxn05456_c0', 'rxn05343_c0', 'rxn05344_c0', 'rxn05408_c0', 'rxn05349_c0', 'rxn05458_c0', 'rxn05421_c0', 'rxn05350_c0', 'rxn05429_c0', 'rxn05453_c0', 'rxn05348_c0', 'rxn05347_c0']
✅ cpd00166_c0 no está en reacciones bloqueadas


Revisamos nuevamente reacciones bloeuqadas en las que participen los precursores de la biomasa y las desbloquamos

In [31]:

# 📌 Metabolito bloqueado y sus reacciones asociadas
blocked_metabolite = "cpd11493_c0"
blocked_reactions =   ['rxn05421_c0', 'rxn05343_c0', 'rxn05429_c0', 'rxn05409_c0', 'rxn05455_c0', 'rxn05453_c0', 'rxn05459_c0', 'rxn05349_c0', 'rxn08433_c0', 'rxn05456_c0', 'rxn05348_c0', 'rxn05350_c0', 'rxn05408_c0', 'EX_cpd11493_b', 'rxn05347_c0', 'rxn08438_c0', 'rxn05425_c0', 'rxn05417_c0', 'rxn05345_c0', 'rxn05451_c0', 'rxn05344_c0', 'rxn05460_c0', 'rxn05346_c0', 'rxn05413_c0', 'rxn05458_c0']

# 🔄 Ajustar restricciones de flujo en reacciones bloqueadas
for rxn_id in blocked_reactions:
    if rxn_id in model.reactions:
        rxn = model.reactions.get_by_id(rxn_id)
        print(f"🔍 Revisando {rxn_id}")

        
        rxn.lower_bound = -1  # Permitir entrada de flujo
        rxn.upper_bound = 1  # Permitir flujo positivo
        print(f"✅ Desbloqueada {rxn_id} (Ajustado lower_bound y upper_bound)")

        # Verificar si el metabolito está en la reacción y ajustar su disponibilidad
        if blocked_metabolite in [met.id for met in rxn.metabolites]:
            print(f"🔄 {blocked_metabolite} está en {rxn_id}, verificando balanceo.")

            # Si es necesario, permitir intercambio si no tiene reacción de intercambio
            exchange_rxn_id = f"EX_{blocked_metabolite.replace('_c0', '_b')}"
            if exchange_rxn_id not in model.reactions:
                exchange_rxn = cobra.Reaction(exchange_rxn_id)
                exchange_rxn.add_metabolites({model.metabolites.get_by_id(blocked_metabolite): -1})
                exchange_rxn.lower_bound = -1  # Permitir consumo
                exchange_rxn.upper_bound = 1  # Mantener flujo abierto
                model.add_reactions([exchange_rxn])
                print(f"✅ Se agregó reacción de intercambio {exchange_rxn_id}")




🔍 Revisando rxn05421_c0
✅ Desbloqueada rxn05421_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05421_c0, verificando balanceo.
🔍 Revisando rxn05343_c0
✅ Desbloqueada rxn05343_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05343_c0, verificando balanceo.
🔍 Revisando rxn05429_c0
✅ Desbloqueada rxn05429_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05429_c0, verificando balanceo.
🔍 Revisando rxn05409_c0
✅ Desbloqueada rxn05409_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05409_c0, verificando balanceo.
🔍 Revisando rxn05455_c0
✅ Desbloqueada rxn05455_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05455_c0, verificando balanceo.
🔍 Revisando rxn05453_c0
✅ Desbloqueada rxn05453_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05453_c0, verificando balanceo.
🔍 Revisando rxn05459_c0
✅ Desbloqueada rxn05459_c0 (Ajustado lower_bound y upper_bound)
🔄 cpd11493_c0 está en rxn05459_c0, verific

In [32]:
# 💾 Guardar el modelo corregido
cobra.io.write_sbml_model(model, "modelo_v13.xml")
print("\n✅ Proceso completado. Modelo guardado como 'modelo_desbloqueado.xml'")



✅ Proceso completado. Modelo guardado como 'modelo_desbloqueado.xml'


### Energy cycles and fluxes

Una vez arregladas las reacciones de biomasa nos encontramos con que el modelo tienen ciclos de energía incorrectos, en estos se produce ATP solo lo que es imposible. Los ciclos de energía incorrectos se producen porque la direccionalidad y los límites de las reacciones no se han establecido correctamente. Para estudiar los límites de flujo de cada reaccion habría que acer experimentos de marcaje enn el laboratorio (no hay estudios hechos para dechloromonas aormatica), nos centramos mas en ajustar la reversibilidad de las reacciones. Para ello lo que hacemos es calcular automáticamente su energía libre de gibbs y en base a eso ajustamos si es o no reversible. 

Obtener identificadores KEGG de las reacciones para calcular energía libre de gibbs

In [33]:
import cobra
import requests
import pandas as pd

# Cargar el modelo metabólico en formato SBML
modelo = cobra.io.read_sbml_model("modelo_v13.xml")

# Lista para almacenar los resultados
resultados = []

# Recorrer todas las reacciones del modelo
for rxn in modelo.reactions:
    # Obtener el identificador KEGG desde las anotaciones
    kegg_id = rxn.annotation.get("kegg.reaction", None)
    
    if kegg_id:
        try:
            # Consultar la ecuación de la reacción en la API de KEGG
            url = f"https://rest.kegg.jp/get/{kegg_id}"
            response = requests.get(url)
            
            if response.status_code == 200:
                data = response.text.split("\n")

                # Buscar la línea donde aparece la ecuación de la reacción
                equation = "No encontrada"
                for line in data:
                    if line.startswith("EQUATION"):
                        equation = line.replace("EQUATION", "").strip()
                        print(equation)
                        break
            else:
                equation = "Error en la consulta"
            
            # Guardar los resultados
            resultados.append({"Reacción": rxn.id, "KEGG ID": kegg_id, "Ecuación": equation})

        except Exception as e:
            resultados.append({"Reacción": rxn.id, "KEGG ID": kegg_id, "Ecuación": f"Error: {str(e)}"})

# Convertir la lista en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Guardar los resultados en un archivo CSV
df_resultados.to_csv("reacciones_kegg_ecuaciones.csv", index=False)

# Mostrar los resultados
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Reacciones y Fórmulas KEGG", dataframe=df_resultados)


C04807 + C00568 <=> C00013 + C00921
C00002 + C00669 + C00037 <=> C00008 + C00009 + C00051
C02715 + C00001 <=> C00060 + C00049
C01290 + C00001 <=> C01190 + C00124
C00065 + C00024 <=> C00979 + C00010
C00002 + C00055 <=> C00008 + C00112
C00630 <=> C00136
C05897 + C00043 <=> C05898 + C00015
C00002 + C00705 <=> C00008 + C00458
C05125 + C15972 <=> C16255 + C00068
C00096 <=> C01222 + C00001
C00208 + C00001 <=> 2 C00031
C00002 + C02191 + C00305 + C00001 <=> C00008 + C00009 + C03516 + 2 C00080
C00158 <=> C00311
C11907 <=> C00688
C00047 <=> C01672 + C00011
C11942 + C00003 + C00001 <=> C11943 + C00004 + C00080
C11638 + C11437 <=> C00627 + C00009 + 2 C00001
C05379 <=> C00026 + C00011
C03201 + C00307 <=> C05212 + C00055
C00002 + C04071 <=> C00008 + C04503
C00002 + C00239 <=> C00008 + C00705
C00081 + C00036 <=> C00104 + C00074 + C00011
C00083 + C00022 <=> C00024 + C00036
C03794 <=> C00122 + C00020
C00063 + C03492 + C00097 <=> C00055 + C00013 + C04352
C00029 + C00001 + 2 C00003 <=> C00167 + 2 C00004 

In [34]:
# Convertir la lista en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Guardar los resultados en un archivo CSV
df_resultados.to_csv("reacciones_kegg_ecuaciones.csv", index=False)

In [35]:
df_resultados

,Reacción,KEGG ID,Ecuación
0,rxn02201_c0,R03067,C04807 + C00568 <=> C00013 + C00921
1,rxn00351_c0,R00497,C00002 + C00669 + C00037 <=> C00008 + C00009 +...
2,rxn00390_c0,R00546,C02715 + C00001 <=> C00060 + C00049
3,rxn11567_c0,R03355,C01290 + C00001 <=> C01190 + C00124
4,rxn00423_c0,R00586,C00065 + C00024 <=> C00979 + C00010
...,...,...,...
798,rxn01152_c0,R01577,C00216 <=> C00309
799,rxn02269_c0,R03171,C00671 + C00010 + C00003 <=> C15980 + C00011 +...
800,rxn05335_c0,R04568,C04688 <=> C05760 + C00001
801,rxn03511_c0,R05176,C06397 <=> C06398


Traducir flechas de las reacciones para que puedan ser parseadas por Equilibrator (calculador de energía libre de gibbs)

In [36]:
import pandas as pd
import re

# Cargar el DataFrame con reacciones y ecuaciones KEGG
df_reacciones = pd.read_csv("reacciones_kegg_ecuaciones.csv")

# Diccionario de reemplazo para flechas comunes en ecuaciones
flechas = {
    "<=>"  : " ↽⇀ ",  # Flecha reversible común en modelos metabólicos
    "<="   : " → ",   # Flecha irreversible
    "=>"   : " → ",   # Flecha irreversible
    "<->"  : " ↽⇀ ",  # Flecha reversible alternativa
    "-->"  : " → ",   # Flecha irreversible alternativa
    "<="   : " → ",   # Flecha irreversible alternativa
    "="    : " → "    # Flecha genérica (si hay problemas con parseo)
}

# Función para reemplazar las flechas en la ecuación de la reacción
def normalizar_flechas(ecuacion):
    if isinstance(ecuacion, str):  # Evitar errores con valores NaN
        for simbolo, flecha_equilibrator in flechas.items():
            ecuacion = ecuacion.replace(simbolo, flecha_equilibrator)
        return ecuacion.strip()  # Eliminar espacios extra
    return ecuacion  # Si el valor no es una cadena, dejarlo igual

# Aplicar la normalización a la columna de ecuaciones
df_reacciones["Ecuación Normalizada"] = df_reacciones["Ecuación"].apply(normalizar_flechas)

# Guardar los resultados en un nuevo archivo CSV
df_reacciones.to_csv("reacciones_kegg_ecuaciones_corregidas.csv", index=False)

# Mostrar los resultados corregidos
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Reacciones con Flechas Normalizadas", dataframe=df_reacciones)


In [48]:
df_reacciones = pd.read_csv("reacciones_kegg_ecuaciones_corregidas.csv")
reactions={}
cc = ComponentContribution()
for index, row in df_reacciones.iterrows():
    rxn_id = row["Reacción"]
    kegg_id = row["KEGG ID"]
    ecuacion = row["Ecuación"]
    if ecuacion != "Error en la consulta":
        print(ecuacion)
        reaction=cc.parse_reaction_formula(ecuacion)
        if isinstance(reaction, list) and len(reaction) > 1:
            print(f"⚠️ Múltiples coincidencias encontradas para {rxn_id}. Seleccionando la primera.")
            reaction = reaction[0] 
        reactions[kegg_id]=reaction
        print(reaction)

C04807 + C00568 <=> C00013 + C00921
Compound(id=400, inchi_key=ALYNCZNDIQEVRV-UHFFFAOYSA-M) + Compound(id=1074, inchi_key=FCQGJGLSOWZZON-UHFFFAOYSA-K) <=> Compound(id=14, inchi_key=XPPKVPWEQAFLFU-UHFFFAOYSA-K) + Compound(id=894, inchi_key=WBFYVDCHGVNRBH-UHFFFAOYSA-N)
C00002 + C00669 + C00037 <=> C00008 + C00009 + C00051
Compound(id=6, inchi_key=ZKHQWZAMYRWXGA-KQYNXXCUSA-J) + Compound(id=31, inchi_key=DHMQDGOQFOQNFH-UHFFFAOYSA-N) + Compound(id=391, inchi_key=RITKHVBHSGLULN-WHFBIAKZSA-M) <=> Compound(id=10, inchi_key=XTWYTFMLZFPYCI-KQYNXXCUSA-K) + Compound(id=12, inchi_key=NBIIXXVUZAFLBC-UHFFFAOYSA-L) + Compound(id=57, inchi_key=RWSXRVCMGQZWBV-WDSKDSINSA-M)
C00130 + C00013 <=> C00262 + C00119
Compound(id=14, inchi_key=XPPKVPWEQAFLFU-UHFFFAOYSA-K) + Compound(id=117, inchi_key=GRSZFWQUAKGDAV-KQYNXXCUSA-L) <=> Compound(id=86, inchi_key=PQGCEDQWHSBAJP-TXICZTDVSA-I) + Compound(id=201, inchi_key=FDGQSTZJBFJUBT-UHFFFAOYSA-N)
C00065 + C00024 <=> C00979 + C00010
Compound(id=23, inchi_key=ZSLZBFCD

MultipleResultsFound: Multiple rows were found when one or none was required

Parsear la formula de la ecuación para obtener los compuestos y también la energía libre de gibbs

In [37]:
import pandas as pd
from equilibrator_api import ComponentContribution

# Cargar el DataFrame con reacciones y ecuaciones KEGG corregidas
df_reacciones = pd.read_csv("reacciones_kegg_ecuaciones_corregidas.csv")

# Inicializar el modelo termodinámico de eQuilibrator
cc = ComponentContribution()

# Diccionario para almacenar las reacciones procesadas
reactions = {}

# Procesar cada reacción
for index, row in df_reacciones.iterrows():
    rxn_id = row["Reacción"]
    kegg_id = row["KEGG ID"]
    ecuacion = row["Ecuación"]

    if ecuacion != "Error en la consulta":
        try:
            print(ecuacion)
            
            # Intentar parsear la ecuación
            reaction = cc.parse_reaction_formula(ecuacion)

            # Verificar si hay múltiples coincidencias
            if isinstance(reaction, list) and len(reaction) > 1:
                print(f"⚠️ Múltiples coincidencias encontradas para {rxn_id}. Seleccionando la primera.")
                reaction = reaction[0]  # Seleccionar la primera coincidencia

            # Guardar la reacción en el diccionario
            reactions[kegg_id] = reaction
            print(reaction)

        except Exception as e:
            print(f"⚠️ Error al procesar la reacción {rxn_id}: {str(e)}")
            continue  # Saltar al siguiente ciclo del bucle sin detener el programa


C04807 + C00568 <=> C00013 + C00921
Compound(id=400, inchi_key=ALYNCZNDIQEVRV-UHFFFAOYSA-M) + Compound(id=1074, inchi_key=FCQGJGLSOWZZON-UHFFFAOYSA-K) <=> Compound(id=14, inchi_key=XPPKVPWEQAFLFU-UHFFFAOYSA-K) + Compound(id=894, inchi_key=WBFYVDCHGVNRBH-UHFFFAOYSA-N)
C00002 + C00669 + C00037 <=> C00008 + C00009 + C00051
Compound(id=6, inchi_key=ZKHQWZAMYRWXGA-KQYNXXCUSA-J) + Compound(id=31, inchi_key=DHMQDGOQFOQNFH-UHFFFAOYSA-N) + Compound(id=391, inchi_key=RITKHVBHSGLULN-WHFBIAKZSA-M) <=> Compound(id=10, inchi_key=XTWYTFMLZFPYCI-KQYNXXCUSA-K) + Compound(id=12, inchi_key=NBIIXXVUZAFLBC-UHFFFAOYSA-L) + Compound(id=57, inchi_key=RWSXRVCMGQZWBV-WDSKDSINSA-M)
C02715 + C00001 <=> C00060 + C00049
Compound(id=5, inchi_key=XLYOFNOQVPJJNP-UHFFFAOYSA-N) + Compound(id=4518, inchi_key=None) <=> Compound(id=44, inchi_key=CKLJMWTZIZZHCS-REOHCLBHSA-M) + Compound(id=69, inchi_key=None)
C01290 + C00001 <=> C01190 + C00124
Compound(id=5, inchi_key=XLYOFNOQVPJJNP-UHFFFAOYSA-N) + Compound(id=256, inchi_ke

Calcular energía libre de gibbs en funcion de los compuestos

In [56]:
for ident,reac in reactions.items(): 
    print(ident)
    dG0_prime = cc.standard_dg_prime(reac)
    dG0_prime_value = dG0_prime.magnitude
    print(f"ΔG'° = {dG0_prime_value}")
    

 R03067
ΔG'° = -62+/-7
 R00497
ΔG'° = -30.7+/-2.6
 R01132
ΔG'° = 30.6+/-1.6
 R00586
ΔG'° = -6.4+/-2.9
 R00512
ΔG'° = 1.8+/-2.9
 R05032
ΔG'° = -15.0+/-3.5
 R03037
ΔG'° = -41.1+/-2.8
 R02326
ΔG'° = 4.1+/-2.4
 R03270
ΔG'° = (-0.0+/-1.0)e+05
 R01324
ΔG'° = 6.85+/-0.33
 R06514
ΔG'° = -15.4+/-1.9
 R04737
ΔG'° = 15.6+/-1.0
 R00462
ΔG'° = -22.2+/-3.5
 R06408
ΔG'° = -44.1+/-1.1
 R05611
ΔG'° = 81.2+/-2.9
 R06785
ΔG'° = -62+/-7
 R05838
ΔG'° = -222+/-4
 R00268
ΔG'° = -10+/-4
 R04388
ΔG'° = 13.5+/-1.4
 R01665
ΔG'° = 1.5+/-2.9
 R00726
ΔG'° = 9.3+/-3.4
 R00353
ΔG'° = 1.9+/-1.5
 R01083
ΔG'° = 16.2+/-1.4
 R04231
ΔG'° = -57+/-5
 R00286
ΔG'° = -28.5+/-2.2
 R00239
ΔG'° = 28.9+/-2.3
 R04218
ΔG'° = 44+/-8
 R00816
ΔG'° = -262+/-6
 R01654
ΔG'° = -38+/-4
 R00989
ΔG'° = -17.9+/-3.4
 R00259
ΔG'° = -25.7+/-3.4
 R04909
ΔG'° = 18.3+/-3.1
 R04150
ΔG'° = -108+/-8
 R02145
ΔG'° = (0+/-7)e+04
 R07092
ΔG'° = (-0.0+/-1.0)e+05
 R03243
ΔG'° = -8.8+/-2.9
 R02770
ΔG'° = (0.0+/-1.0)e+05
 R04972
ΔG'° = -22+/-12
 R03504
ΔG'° = 7

Ajustar reversibilidad según energía libre de gibbs

In [60]:
import cobra
import pandas as pd
from equilibrator_api import ComponentContribution

# Cargar el modelo metabólico en SBML
modelo = cobra.io.read_sbml_model("modelo_v13.xml")

# Inicializar el modelo termodinámico de eQuilibrator
cc = ComponentContribution()

# Lista para almacenar los resultados
resultados = []

# Iterar sobre cada reacción en el diccionario
for kegg_id, reac in reactions.items():
    try:
        # Calcular ΔG'° estándar
        dG0_prime = cc.standard_dg_prime(reac).magnitude

        # Determinar los límites de flujo en base a ΔG'°
        if abs(dG0_prime) < 30:
            lb, ub = -1000, 1000  # Reacción reversible
            print("ajustada a reversible")
        elif dG0_prime < -30:
            lb, ub = 0, 1000  # Irreversible hacia adelante
            print("ajustada a ireversible")
        else:
            lb, ub = -1000, 0  # Irreversible hacia atrás
            print("ajustada a ireversible")

        # Aplicar restricciones al modelo en COBRApy
        if kegg_id in modelo.reactions:
            modelo.reactions.get_by_id(kegg_id).lower_bound = lb
            print("ajustada a reversible1")
            modelo.reactions.get_by_id(kegg_id).upper_bound = ub
            print("ajustada a ireversible2")

        # Guardar los resultados
        resultados.append({
            "KEGG ID": kegg_id, 
            "ΔG'° (kJ/mol)": dG0_prime, 
            "Lower Bound": lb, 
            "Upper Bound": ub
        })

        print(f"✅ {kegg_id} -> ΔG'°: {dG0_prime:.2f} kJ/mol | Bounds: [{lb}, {ub}]")

    except Exception as e:
        print(f"⚠️ Error al calcular ΔG'° para {kegg_id}: {str(e)}")
        resultados.append({
            "KEGG ID": kegg_id, 
            "ΔG'° (kJ/mol)": f"Error: {str(e)}", 
            "Lower Bound": "N/A", 
            "Upper Bound": "N/A"
        })
        continue  # Saltar al siguiente ciclo del bucle sin detener el programa

# Convertir los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)


# Guardar el modelo corregido con las restricciones de flujo ajustadas
cobra.io.write_sbml_model(modelo, "modelo_v14.xml")



ajustada a ireversible
✅  R03067 -> ΔG'°: -62.42+/-6.52 kJ/mol | Bounds: [0, 1000]
ajustada a ireversible
✅  R00497 -> ΔG'°: -30.66+/-2.62 kJ/mol | Bounds: [0, 1000]
ajustada a ireversible
✅  R01132 -> ΔG'°: 30.57+/-1.58 kJ/mol | Bounds: [-1000, 0]
ajustada a reversible
✅  R00586 -> ΔG'°: -6.40+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R00512 -> ΔG'°: 1.81+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R05032 -> ΔG'°: -15.03+/-3.49 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R03037 -> ΔG'°: -41.14+/-2.83 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R02326 -> ΔG'°: 4.06+/-2.41 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R03270 -> ΔG'°: -174.39+/-100000.00 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R01324 -> ΔG'°: 6.85+/-0.33 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R06514 -> ΔG'°: -15.40+/-1.90 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R04737 -> ΔG'°: 15.62+/-0.99 kJ/mol | Bounds:

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=74442, inchi_key=None): Compound(id=74442, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=5218, inchi_key=None): Compound(id=5218, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=2684, inchi_key=None): Compound(id=2684, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=4516, i

ajustada a ireversible
✅  R02061 -> ΔG'°: -81.25+/-2.94 kJ/mol | Bounds: [0, 1000]
ajustada a ireversible
✅  R01209 -> ΔG'°: -36.93+/-1.40 kJ/mol | Bounds: [0, 1000]
ajustada a ireversible
✅  R07281 -> ΔG'°: -88.20+/-2.06 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R02282 -> ΔG'°: 2.09+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R02922 -> ΔG'°: 14.30+/-6.97 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R08280 -> ΔG'°: 0.03+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R04292 -> ΔG'°: 219.74+/-4.24 kJ/mol | Bounds: [-1000, 0]
ajustada a ireversible
✅  R07003 -> ΔG'°: 353.40+/-62806.54 kJ/mol | Bounds: [-1000, 0]
ajustada a reversible
✅  R00334 -> ΔG'°: 8.74+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R00749 -> ΔG'°: -35.94+/-3.55 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R00930 -> ΔG'°: -2.17+/-5.26 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R05366 -> ΔG'°: -64.65+/-2.60 kJ/mol |

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=1501, inchi_key=None): Compound(id=1501, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=925, inchi_key=None): Compound(id=925, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=2342, inchi_key=None): Compound(id=2342, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=174, inchi_

ajustada a ireversible
✅  R04230 -> ΔG'°: -55.61+/-4.65 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R01213 -> ΔG'°: 26.48+/-1.35 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R04385 -> ΔG'°: 5.70+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R04247 -> ΔG'°: -45.58+/-100000.00 kJ/mol | Bounds: [0, 1000]
ajustada a ireversible
✅  R00897 -> ΔG'°: -51.28+/-5.24 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R01728 -> ΔG'°: -10.78+/-11.29 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R02119 -> ΔG'°: -25.72+/-1.94 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R00885 -> ΔG'°: -8.49+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R02735 -> ΔG'°: -1.65+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R00336 -> ΔG'°: -26.53+/-6.32 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R07023 -> ΔG'°: 0.04+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R00787 -> ΔG'°: 378.72+/-3.6

...

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=14285, inchi_key=None): Compound(id=14285, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=1129, inchi_key=None): Compound(id=1129, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=1128, inchi_key=None): Compound(id=1128, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=2173, i

ajustada a reversible
✅  R07034 -> ΔG'°: -1.44+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R00904 -> ΔG'°: -46.56+/-6.01 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R07083 -> ΔG'°: 0.02+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R01248 -> ΔG'°: 44.49+/-2.95 kJ/mol | Bounds: [-1000, 0]
ajustada a ireversible
✅  R08951 -> ΔG'°: -83.64+/-3.27 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R08381 -> ΔG'°: -7.45+/-2.16 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R03024 -> ΔG'°: -5.14+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R00351 -> ΔG'°: 38.76+/-0.44 kJ/mol | Bounds: [-1000, 0]
ajustada a reversible
✅  R04325 -> ΔG'°: -1.44+/-5.69 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R00189 -> ΔG'°: -23.61+/-2.80 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R04567 -> ΔG'°: 1133.52+/-100000.00 kJ/mol | Bounds: [-1000, 0]
ajustada a reversible
✅  R01238 -> ΔG'°: -7.74+/-3.32

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=26, inchi_key=None): Compound(id=26, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=7263, inchi_key=None): Compound(id=7263, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=16272, inchi_key=None): Compound(id=16272, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(


ajustada a ireversible
✅  R08583 -> ΔG'°: 40.37+/-100000.00 kJ/mol | Bounds: [-1000, 0]
ajustada a reversible
✅  R02094 -> ΔG'°: -1.66+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R03858 -> ΔG'°: 30.17+/-1.34 kJ/mol | Bounds: [-1000, 0]
ajustada a reversible
✅  R02938 -> ΔG'°: -13.68+/-2.28 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R01736 -> ΔG'°: -33.98+/-3.03 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R03458 -> ΔG'°: 25.28+/-1.68 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R04745 -> ΔG'°: 15.62+/-0.99 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R00835 -> ΔG'°: -6.82+/-1.32 kJ/mol | Bounds: [-1000, 1000]
ajustada a reversible
✅  R00925 -> ΔG'°: -1.15+/-2.93 kJ/mol | Bounds: [-1000, 1000]
ajustada a ireversible
✅  R01398 -> ΔG'°: -30.31+/-2.93 kJ/mol | Bounds: [0, 1000]
ajustada a ireversible
✅  R04089 -> ΔG'°: -262.16+/-5.69 kJ/mol | Bounds: [0, 1000]
ajustada a reversible
✅  R08706 -> ΔG'°: 0.00+/-0 kJ/mol | Bounds: [

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=74705, inchi_key=None): Compound(id=74705, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(


In [38]:
import cobra
import pandas as pd
from equilibrator_api import ComponentContribution

# Cargar el modelo metabólico en SBML
modelo = cobra.io.read_sbml_model("modelo_v13.xml")

# Inicializar el modelo termodinámico de eQuilibrator
cc = ComponentContribution()

# Lista para almacenar los resultados
resultados = []

# Función para encontrar la reacción en el modelo a partir de KEGG ID
def encontrar_reaccion_por_kegg(kegg_id, modelo):
    for reaction in modelo.reactions:
        if "kegg.reaction" in reaction.annotation and reaction.annotation["kegg.reaction"] == kegg_id:
            return reaction
    return None  # No encontrada

# Iterar sobre cada reacción en el diccionario
for kegg_id, reac in reactions.items():
    try:
        # Calcular ΔG'° estándar
        dG0_prime = cc.standard_dg_prime(reac)

        # Extraer solo el valor numérico (sin unidades ni incertidumbre)
        dG0_prime_value = dG0_prime.magnitude

        # Determinar los límites de flujo en base a ΔG'°
        if abs(dG0_prime_value) < 30:
            lb, ub = -1000, 1000  # Reacción reversible
        elif dG0_prime_value < -30:
            lb, ub = 0, 1000  # Irreversible hacia adelante
        else:
            lb, ub = -1000, 0  # Irreversible hacia atrás

        # Encontrar la reacción en el modelo metabólico usando KEGG ID
        reaction_obj = encontrar_reaccion_por_kegg(kegg_id, modelo)

        # Aplicar restricciones al modelo en COBRApy si la reacción fue encontrada
        if reaction_obj:
            reaction_obj.lower_bound = lb
            reaction_obj.upper_bound = ub
            print(f"✅ {reaction_obj.id} (KEGG: {kegg_id}) -> ΔG'°: {dG0_prime_value:.2f} kJ/mol | Bounds: [{lb}, {ub}]")
        else:
            print(f"⚠️ No se encontró reacción en el modelo para KEGG ID {kegg_id}")

        # Guardar los resultados
        resultados.append({
            "KEGG ID": kegg_id, 
            "Reacción en Modelo": reaction_obj.id if reaction_obj else "No encontrada",
            "ΔG'° (kJ/mol)": dG0_prime_value, 
            "Lower Bound": lb, 
            "Upper Bound": ub
        })

    except Exception as e:
        print(f"⚠️ Error al calcular ΔG'° para {kegg_id}: {str(e)}")
        resultados.append({
            "KEGG ID": kegg_id, 
            "Reacción en Modelo": "Error",
            "ΔG'° (kJ/mol)": f"Error: {str(e)}", 
            "Lower Bound": "N/A", 
            "Upper Bound": "N/A"
        })
        continue  # Saltar al siguiente ciclo del bucle sin detener el programa

# Convertir los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Guardar los resultados en un CSV
df_resultados.to_csv("ajuste_termico_reacciones_equilibrator_a.csv", index=False)

# Guardar el modelo corregido con las restricciones de flujo ajustadas
cobra.io.write_sbml_model(modelo, "modelo_v14.xml")


✅ rxn02201_c0 (KEGG:  R03067) -> ΔG'°: -62.42+/-6.52 kJ/mol | Bounds: [0, 1000]
✅ rxn00351_c0 (KEGG:  R00497) -> ΔG'°: -30.66+/-2.62 kJ/mol | Bounds: [0, 1000]


C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=4518, inchi_key=None): Compound(id=4518, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=69, inchi_key=None): Compound(id=69, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(


✅ rxn00390_c0 (KEGG:  R00546) -> ΔG'°: 173.47+/-100000.00 kJ/mol | Bounds: [-1000, 0]


C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=256, inchi_key=None): Compound(id=256, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=117292, inchi_key=None): Compound(id=117292, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(


✅ rxn11567_c0 (KEGG:  R03355) -> ΔG'°: 435.97+/-100000.00 kJ/mol | Bounds: [-1000, 0]
✅ rxn00423_c0 (KEGG:  R00586) -> ΔG'°: -6.40+/-2.93 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00364_c0 (KEGG:  R00512) -> ΔG'°: 1.81+/-2.93 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00877_c0 (KEGG:  R01181) -> ΔG'°: -2.80+/-0.94 kJ/mol | Bounds: [-1000, 1000]
✅ rxn03408_c0 (KEGG:  R05032) -> ΔG'°: -15.03+/-3.49 kJ/mol | Bounds: [-1000, 1000]
✅ rxn01673_c0 (KEGG:  R02326) -> ΔG'°: 4.06+/-2.41 kJ/mol | Bounds: [-1000, 1000]


...

✅ rxn06759_c0 (KEGG:  R04649) -> ΔG'°: -741.25+/-100000.00 kJ/mol | Bounds: [0, 1000]
✅ rxn00224_c0 (KEGG:  R00310) -> ΔG'°: 3.24+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
✅ rxn01513_c0 (KEGG:  R02094) -> ΔG'°: -1.66+/-2.93 kJ/mol | Bounds: [-1000, 1000]
✅ rxn06510_c0 (KEGG:  R03858) -> ΔG'°: 30.17+/-1.34 kJ/mol | Bounds: [-1000, 0]
✅ rxn11612_c0 (KEGG:  R04633) -> ΔG'°: 435.97+/-100000.00 kJ/mol | Bounds: [-1000, 0]
✅ rxn00209_c0 (KEGG:  R00278) -> ΔG'°: -114.37+/-4.23 kJ/mol | Bounds: [0, 1000]
✅ rxn01274_c0 (KEGG:  R01736) -> ΔG'°: -33.98+/-3.03 kJ/mol | Bounds: [0, 1000]
✅ rxn02474_c0 (KEGG:  R03458) -> ΔG'°: 25.28+/-1.68 kJ/mol | Bounds: [-1000, 1000]
✅ rxn02144_c0 (KEGG:  R02991) -> ΔG'°: -24.65+/-4.05 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00604_c0 (KEGG:  R00835) -> ΔG'°: -6.82+/-1.32 kJ/mol | Bounds: [-1000, 1000]
✅ rxn01019_c0 (KEGG:  R01398) -> ΔG'°: -30.31+/-2.93 kJ/mol | Bounds: [0, 1000]
✅ rxn00633_c0 (KEGG:  R00874) -> ΔG'°: -15.99+/-1.50 kJ/mol | Bounds: [-1000, 1000]
✅ r

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=16272, inchi_key=None): Compound(id=16272, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(


✅ rxn05343_c0 (KEGG:  R04960) -> ΔG'°: 0.00+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
✅ rxn01520_c0 (KEGG:  R02101) -> ΔG'°: -45.28+/-2.94 kJ/mol | Bounds: [0, 1000]
✅ rxn01641_c0 (KEGG:  R02287) -> ΔG'°: 10.79+/-2.07 kJ/mol | Bounds: [-1000, 1000]
✅ rxn11663_c0 (KEGG:  R05837) -> ΔG'°: -1.69+/-3.13 kJ/mol | Bounds: [-1000, 1000]
✅ rxn01226_c0 (KEGG:  R01677) -> ΔG'°: -2.58+/-2.71 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00493_c0 (KEGG:  R00694) -> ΔG'°: -0.08+/-2.07 kJ/mol | Bounds: [-1000, 1000]
✅ rxn01834_c0 (KEGG:  R02530) -> ΔG'°: 23.46+/-2.93 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00293_c0 (KEGG:  R00416) -> ΔG'°: -10.76+/-2.94 kJ/mol | Bounds: [-1000, 1000]
✅ rxn02959_c0 (KEGG:  R04237) -> ΔG'°: 3.22+/-100000.00 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00839_c0 (KEGG:  R01137) -> ΔG'°: 2.39+/-1.47 kJ/mol | Bounds: [-1000, 1000]
✅ rxn03181_c0 (KEGG:  R04657) -> ΔG'°: -11.74+/-1.39 kJ/mol | Bounds: [-1000, 1000]
✅ rxn06865_c0 (KEGG:  R05146) -> ΔG'°: 959.13+/-100000.00 kJ/mol | Bounds: [-1000

C:\Users\elena\anaconda3\envs\nuevo_entorno\lib\site-packages\equilibrator_cache\reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=74705, inchi_key=None): Compound(id=74705, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(


✅ rxn06023_c0 (KEGG:  R01625) -> ΔG'°: -919.92+/-100000.00 kJ/mol | Bounds: [0, 1000]
✅ rxn03919_c0 (KEGG:  R05647) -> ΔG'°: -8.65+/-1.25 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00467_c0 (KEGG:  R00667) -> ΔG'°: -4.26+/-4.52 kJ/mol | Bounds: [-1000, 1000]
✅ rxn03130_c0 (KEGG:  R04549) -> ΔG'°: -32.58+/-3.36 kJ/mol | Bounds: [0, 1000]
✅ rxn01152_c0 (KEGG:  R01577) -> ΔG'°: 5.31+/-0.74 kJ/mol | Bounds: [-1000, 1000]
✅ rxn02269_c0 (KEGG:  R03171) -> ΔG'°: -22.32+/-3.35 kJ/mol | Bounds: [-1000, 1000]
✅ rxn05335_c0 (KEGG:  R04568) -> ΔG'°: 87.19+/-100000.00 kJ/mol | Bounds: [-1000, 0]
✅ rxn03511_c0 (KEGG:  R05176) -> ΔG'°: 0.00+/-0 kJ/mol | Bounds: [-1000, 1000]
✅ rxn00966_c0 (KEGG:  R01302) -> ΔG'°: 112.94+/-7.38 kJ/mol | Bounds: [-1000, 0]


In [39]:
import cobra

# Cargar el modelo SBML
modelo_path = "modelo_v14.xml"  # Asegúrate de que el archivo está en el mismo directorio
model = cobra.io.read_sbml_model(modelo_path)

def get_reaction_equations(reaction_ids):
    equations = {}
    for rid in reaction_ids:
        try:
            reaction = model.reactions.get_by_id(rid)
            equation = reaction.build_reaction_string(use_metabolite_names=True)
            equations[rid] = equation
        except KeyError:
            equations[rid] = "Reacción no encontrada en el modelo."
    return equations

# Lista de identificadores de reacciones a consultar (Ejemplo: primeras 10 reacciones)
reaction_ids_to_check = ["EX_cpd10516_b","EX_cpd00268_b","EX_cpd01080_b","EX_cpd00039_b","EX_cpd00793_b","rxn00423_c0","rxn00364_c0","rxn00973_c0","rxn04070_c0","rxn00199_c0","rxn09240_c0","rxn00519_c0","rxn00800_c0","rxn05367_c0","rxn02504_c0","rxn00262_c0","rxn00898_c0","rxn02988_c0","rxn00678_c0","rxn00001_c0","rxn00710_c0","rxn10254_c0","rxn05459_c0","rxn05434_c0","rxn05733_c0","rxn00416_c0","rxn12512_c0","rxn06672_c0","rxn00225_c0","rxn00148_c0","rxn01102_c0","rxn05569_c0","rxn05358_c0","rxn00100_c0","rxn02374_c0","rxn05560_c0","rxn05437_c0","rxn00781_c0","rxn05319_c0","rxn05298_c0","rxn01355_c0","rxn01451_c0","rxn00097_c0","rxn00695_c0","rxn00692_c0","rxn00178_c0","rxn01362_c0","rxn10125_c0","rxn00549_c0","rxn02933_c0","rxn02268_c0","rxn00305_c0","rxn00612_c0","rxn05466_c0","rxn02380_c0","rxn05938_c0","rxn00471_c0","rxn03848_c0","rxn05435_c0","rxn00907_c0","rxn00336_c0","rxn01465_c0","rxn00747_c0","rxn10113_c0","rxn06673_c0","rxn05256_c0","rxn02175_c0","rxn01454_c0","rxn05359_c0","rxn00555_c0","rxn00147_c0","rxn00251_c0","rxn00237_c0","rxn00285_c0","rxn05371_c0","rxn08615_c0","rxn01015_c0","rxn00299_c0","rxn00904_c0","rxn01281_c0","rxn00786_c0","rxn01100_c0","rxn00838_c0","rxn00512_c0","rxn01211_c0","rxn08527_c0","rxn00173_c0","rxn08528_c0","rxn01669_c0","rxn10122_c0","rxn00187_c0","rxn05508_c0","rxn01116_c0","rxn00868_c0","rxn00908_c0","rxn02402_c0","rxn05452_c0","rxn01870_c0","rxn00113_c0","rxn01678_c0","rxn00872_c0","rxn01106_c0","rxn00412_c0","rxn03167_c0","rxn00515_c0","rxn00313_c0","rxn05363_c0","rxn11040_c0","rxn10042_c0","rxn02934_c0","rxn00189_c0","rxn02508_c0","rxn01996_c0","rxn10253_c0","rxn00283_c0","rxn00726_c0","rxn00379_c0","rxn02155_c0","rxn01018_c0","rxn00947_c0","rxn01476_c0","rxn00988_c0","rxn00410_c0","rxn01333_c0","rxn12510_c0","rxn00333_c0","rxn00414_c0","rxn00799_c0","rxn01977_c0","rxn00011_c0","rxn02200_c0","rxn00602_c0","rxn00791_c0","rxn01672_c0","rxn05312_c0","rxn00105_c0","rxn02285_c0","rxn00691_c0","rxn00290_c0","rxn00459_c0","rxn00770_c0","rxn05436_c0","rxn05195_c0","rxn05209_c0","rxn08444_c0","rxn00461_c0","rxn00669_c0","rxn01387_c0","rxn01115_c0","rxn00161_c0","rxn00371_c0","rxn10226_c0","rxn05379_c0","rxn08655_c0","rxn03841_c0","rxn00676_c0","rxn01257_c0","rxn00943_c0","rxn00903_c0","rxn05226_c0","rxn00874_c0","rxn00834_c0","rxn00785_c0","rxn03909_c0","rxn10806_c0","rxn00346_c0","rxn01790_c0","rxn05651_c0","rxn03638_c0","rxn05438_c0","rxn10043_c0","rxn00991_c0","rxn05375_c0","rxn01200_c0","rxn00438_c0","rxn02185_c0","rxn05176_c0","rxn00912_c0","rxn03849_c0","rxn08094_c0","rxn05433_c0","rxn00338_c0","rxn00085_c0","rxn00119_c0","rxn05468_c0","rxn05740_c0","rxn00409_c0","rxn00193_c0","rxn00256_c0","rxn00138_c0","rxn00347_c0","rxn02167_c0","rxn10161_c0","rxn01485_c0","rxn00239_c0","rxn01791_c0","rxn00623_c0","rxn02507_c0","rxn05937_c0","rxn10220_c0","rxn02341_c0","rxn01014_c0","rxn00224_c0","rxn00604_c0","rxn05605_c0","rxn00674_c0","rxn05305_c0","rxn01603_c0","rxn00686_c0","rxn00905_c0","rxn00157_c0","rxn01286_c0","rxn05289_c0","rxn05467_c0","rxn00117_c0","rxn00330_c0","rxn00560_c0","rxn00293_c0","rxn02011_c0","rxn05361_c0","rxn02008_c0","rxn01361_c0","rxn05377_c0","rxn02304_c0","rxn05369_c0","rxn05380_c0","rxn02186_c0","rxn05365_c0","rxn05368_c0","rxn00851_c0","rxn05360_c0","rxn05376_c0","rxn05381_c0","rxn05722_c0","rxn00467_c0","rxn02286_c0","rxn00916_c0","rxn12663_c0","rxn00056_c0","rxn10206_c0","rxn05372_c0","rxn05373_c0","rxn05364_c0","rxn10215_c0","rxn03164_c0","EX_cpd00067_e0","EX_cpd00276_e0","EX_cpd00082_e0","EX_cpd00001_e0","EX_cpd00023_e0","EX_cpd00013_e0","EX_cpd00132_e0","EX_cpd00009_e0","EX_cpd00048_e0","EX_cpd10516_e0","EX_cpd00027_e0","EX_cpd00268_e0","EX_cpd00007_e0","EX_cpd01080_e0","EX_cpd00130_e0","EX_cpd00039_e0","EX_cpd00011_e0","EX_cpd00793_e0","EX_cpd00038_e0"]
              

# Obtener las ecuaciones de las reacciones especificadas
reaction_equations = get_reaction_equations(reaction_ids_to_check)

# Imprimir los resultados
for rid, eq in reaction_equations.items():
    print(f"{rid}: {eq}")


EX_cpd10516_b: fe3_e0_b <=> 
EX_cpd00268_b: H2S2O3_e0_b <=> 
EX_cpd01080_b: ocdca_e0_b <=> 
EX_cpd00039_b: Reacción no encontrada en el modelo.
EX_cpd00793_b: Thiamine_phosphate_e0_b <=> 
rxn00423_c0: Acetyl_CoA_c0 + L_Serine_c0 <=> CoA_c0 + O_Acetyl_L_serine_c0
rxn00364_c0: ATP_c0 + CMP_c0 + H_plus__c0 <=> ADP_c0 + CDP_c0
rxn00973_c0: Citrate_c0 <=> Isocitrate_c0
rxn04070_c0: 1_deoxy_D_xylulose5_phosphate_c0 + 3_Amino_2_oxopropyl_phosphate_c0 --> 2.0 H2O_c0 + Phosphate_c0 + H_plus__c0 + Pyridoxine_phosphate_c0
rxn00199_c0: H_plus__c0 + Oxalosuccinate_c0 <=> CO2_c0 + 2_Oxoglutarate_c0
rxn09240_c0: Reacción no encontrada en el modelo.
rxn00519_c0: Oxaloacetate_c0 + ITP_c0 <=> CO2_c0 + Phosphoenolpyruvate_c0 + IDP_c0
rxn00800_c0: Adenylosuccinate_c0 <=> AMP_c0 + Fumarate_c0
rxn05367_c0: Malonyl_acyl_carrierprotein__c0 + 6_methyl_octanoyl_ACP_c0 <=> CO2_c0 + ACP_c0 + 8_methyl_3_oxo_decanoyl_ACP_c0
rxn02504_c0: Dihydroneopterin_c0 <=> Glycolaldehyde_c0 + 6_hydroxymethyl_dihydropterin_c0
rx

In [16]:
model=cobra.io.read_sbml_model("modelo_v13.xml")
list_reactions=model=cobra.io.read_sbml_model("modelo_v13.xml")
list_reactions= ["EX_cpd15302_b","EX_cpd00036_b","EX_cpd00082_b","EX_cpd00023_b","EX_cpd00058_b","EX_cpd00034_b","EX_cpd00132_b","EX_cpd00254_b","EX_cpd00205_b","EX_cpd00009_b","EX_cpd10516_b","EX_cpd03847_b","EX_cpd00149_b","EX_cpd00099_b","EX_cpd00268_b","EX_cpd01080_b","EX_cpd00039_b","EX_cpd00011_b","EX_cpd00241_b","EX_cpd15605_b","EX_cpd00793_b","EX_cpd00030_b","EX_cpd00264_b","EX_cpd00111_b","EX_cpd00038_b","EX_cpd00028_b","EX_cpd00063_b","EX_cpd00322_b","EX_cpd11606_b","EX_cpd00115_b","EX_cpd00218_b","rxn00423_c0","rxn00364_c0","rxn03408_c0","rxn05440_c0","rxn00973_c0","rxn02000_c0","rxn05457_c0","rxn05392_c0","rxn12008_c0","rxn00199_c0","rxn09240_c0","rxn00519_c0","rxn00258_c0","rxn00800_c0","rxn00179_c0","rxn05367_c0","rxn05654_c0","rxn02320_c0","rxn02504_c0","rxn04996_c0","rxn00262_c0","rxn05404_c0","rxn01486_c0","rxn00898_c0","rxn05444_c0","rxn05040_c0","rxn05400_c0","rxn02988_c0","rxn00678_c0","rxn00710_c0","rxn00470_c0","rxn00910_c0","rxn03181_c0","rxn02331_c0","rxn06493_c0","rxn12846_c0","rxn00337_c0","rxn05434_c0","rxn05733_c0","rxn12512_c0","rxn00902_c0","rxn06672_c0","rxn00225_c0","rxn00148_c0","rxn01102_c0","rxn05569_c0","rxn05358_c0","rxn00100_c0","rxn00789_c0","rxn02374_c0","rxn05560_c0","rxn05437_c0","rxn00781_c0","rxn05298_c0","rxn09202_c0","rxn00097_c0","rxn02212_c0","rxn00695_c0","rxn10481_c0","rxn00692_c0","rxn00802_c0","rxn02160_c0","rxn01997_c0","rxn03958_c0","rxn00178_c0","rxn01362_c0","rxn02933_c0","rxn00727_c0","rxn02268_c0","rxn00612_c0","rxn02159_c0","rxn01111_c0","rxn05466_c0","rxn00300_c0","rxn05938_c0","rxn06078_c0","rxn03848_c0","rxn05435_c0","rxn11946_c0","rxn01504_c0","rxn00907_c0","rxn02404_c0","rxn05443_c0","rxn03891_c0","rxn01465_c0","rxn01643_c0","rxn00248_c0","rxn01269_c0","rxn00392_c0","rxn00747_c0","rxn10113_c0","rxn05256_c0","rxn02175_c0","rxn02835_c0","rxn01675_c0","rxn03536_c0","rxn05150_c0","rxn05359_c0","rxn00555_c0","rxn03080_c0","rxn00849_c0","rxn00213_c0","rxn01740_c0","rxn00251_c0","rxn03910_c0","rxn00237_c0","rxn08311_c0","rxn03175_c0","rxn10227_c0","rxn09210_c0","rxn00285_c0","rxn05371_c0","rxn08615_c0","rxn00126_c0","rxn08618_c0","rxn00058_c0","rxn00299_c0","rxn00904_c0","rxn03159_c0","rxn01000_c0","rxn01300_c0","rxn03908_c0","rxn01100_c0","rxn00838_c0","rxn05396_c0","rxn01255_c0","rxn01211_c0","rxn00527_c0","rxn03537_c0","rxn00966_c0","rxn00173_c0","rxn08528_c0","rxn01669_c0","rxn10122_c0","rxn09106_c0","rxn05508_c0","rxn03135_c0","rxn10265_c0","rxn00915_c0","rxn01116_c0","rxn02811_c0","rxn03062_c0","rxn01544_c0","rxn00863_c0","rxn09113_c0","rxn08711_c0","rxn00018_c0","rxn02402_c0","rxn05452_c0","rxn05465_c0","rxn05388_c0","rxn01870_c0","rxn01303_c0","rxn00113_c0","rxn01678_c0","rxn00832_c0","rxn01106_c0","rxn03167_c0","rxn00515_c0","rxn00616_c0","rxn06723_c0","rxn00313_c0","rxn05363_c0","rxn11040_c0","rxn10168_c0","rxn10042_c0","rxn02934_c0","rxn02508_c0","rxn05442_c0","rxn01996_c0","rxn00420_c0","rxn10221_c0","rxn10266_c0","rxn05005_c0","rxn00548_c0","rxn00048_c0","rxn00283_c0","rxn00726_c0","rxn00379_c0","rxn02866_c0","rxn02475_c0","rxn01018_c0","rxn00278_c0","rxn01476_c0","rxn03146_c0","rxn03893_c0","rxn02789_c0","rxn00988_c0","rxn00410_c0","rxn01333_c0","rxn12510_c0","rxn00414_c0","rxn00799_c0","rxn02405_c0","rxn00011_c0","rxn02200_c0","rxn00602_c0","rxn01069_c0","rxn00791_c0","rxn03150_c0","rxn01672_c0","rxn05312_c0","rxn00105_c0","rxn01302_c0","rxn05029_c0","rxn02285_c0","rxn02303_c0","rxn02889_c0","rxn00290_c0","rxn03907_c0","rxn03538_c0","rxn02834_c0","rxn01213_c0","rxn00459_c0","rxn00770_c0","rxn05436_c0","rxn05195_c0","rxn05209_c0","rxn06377_c0","rxn01208_c0","rxn10163_c0","rxn10474_c0","rxn00461_c0","rxn10473_c0","rxn05287_c0","rxn01387_c0","rxn03137_c0","rxn00952_c0","rxn01517_c0","rxn10226_c0","rxn05006_c0","rxn05379_c0","rxn02476_c0","rxn01512_c0","rxn03841_c0","rxn06865_c0","rxn04113_c0","rxn00676_c0","rxn01257_c0","rxn09272_c0","rxn00903_c0","rxn02213_c0","rxn05441_c0","rxn03892_c0","rxn05454_c0","rxn05226_c0","rxn00615_c0","rxn03393_c0","rxn00083_c0","rxn00834_c0","rxn00782_c0","rxn01187_c0","rxn00704_c0","rxn01332_c0","rxn00693_c0","rxn03909_c0","rxn10233_c0","rxn06600_c0","rxn00558_c0","rxn00346_c0","rxn00806_c0","rxn05323_c0","rxn05616_c0","rxn01790_c0","rxn05651_c0","rxn00077_c0","rxn03638_c0","rxn05438_c0","rxn02897_c0","rxn10043_c0","rxn00991_c0","rxn10260_c0","rxn00134_c0","rxn00141_c0","rxn02003_c0","rxn05375_c0","rxn01200_c0","rxn00438_c0","rxn02185_c0","rxn00763_c0","rxn05176_c0","rxn02473_c0","rxn06060_c0","rxn01434_c0","rxn00912_c0","rxn03849_c0","rxn00122_c0","rxn01011_c0","rxn10259_c0","rxn08094_c0","rxn05433_c0","rxn00338_c0","rxn05384_c0","rxn02373_c0","rxn00119_c0","rxn05468_c0","rxn09345_c0","rxn01466_c0","rxn05740_c0","rxn03904_c0","rxn01225_c0","rxn02288_c0","rxn10199_c0","rxn00409_c0","rxn03901_c0","rxn10052_c0","rxn00929_c0","rxn00256_c0","rxn10232_c0","rxn06729_c0","rxn00347_c0","rxn10161_c0","rxn01485_c0","rxn00239_c0","rxn03130_c0","rxn01791_c0","rxn00623_c0","rxn02507_c0","rxn05937_c0","rxn10220_c0","rxn01964_c0","rxn02341_c0","rxn03511_c0","rxn01014_c0","rxn00224_c0","rxn01513_c0","rxn02474_c0","rxn00604_c0","rxn05605_c0","rxn00674_c0","rxn01019_c0","rxn05439_c0","rxn05305_c0","rxn02914_c0","rxn01603_c0","rxn00686_c0","rxn00157_c0","rxn05289_c0","rxn05467_c0","rxn00117_c0","rxn01520_c0","rxn03395_c0","rxn05736_c0","rxn00493_c0","rxn00293_c0","rxn05383_c0","rxn05039_c0","rxn08712_c0","rxn02011_c0","rxn05361_c0","rxn02008_c0","rxn01361_c0","rxn05235_c0","rxn05377_c0","rxn08709_c0","rxn05406_c0","rxn12224_c0","rxn08954_c0","rxn02304_c0","rxn05369_c0","rxn05144_c0","rxn03439_c0","rxn10338_c0","rxn05380_c0","rxn05390_c0","rxn05398_c0","rxn12225_c0","rxn03916_c0","rxn04139_c0","rxn08620_c0","rxn10205_c0","rxn05405_c0","rxn12648_c0","rxn05389_c0","rxn02186_c0","rxn09225_c0","rxn06848_c0","rxn05365_c0","rxn03182_c0","rxn10094_c0","rxn13661_c0","rxn05368_c0","rxn03918_c0","rxn00851_c0","rxn05360_c0","rxn05376_c0","rxn10336_c0","rxn09436_c0","rxn05386_c0","rxn12850_c0","rxn05381_c0","rxn05722_c0","rxn00467_c0","rxn08710_c0","rxn02286_c0","rxn05618_c0","rxn00916_c0","rxn05394_c0","rxn02774_c0","rxn03917_c0","rxn05175_c0","rxn05267_c0","rxn05335_c0","rxn08713_c0","rxn09677_c0","rxn06023_c0","rxn03919_c0","rxn12663_c0","rxn00137_c0","rxn00056_c0","rxn01101_c0","rxn05385_c0","rxn10206_c0","rxn05148_c0","rxn08583_c0","rxn05293_c0","rxn05514_c0","rxn05372_c0","rxn01739_c0","rxn05402_c0","rxn08822_c0","rxn05373_c0","rxn05244_c0","rxn01117_c0","rxn05393_c0","rxn03397_c0","rxn08333_c0","rxn12664_c0","rxn08708_c0","rxn05364_c0","rxn05397_c0","rxn12647_c0","rxn05401_c0","rxn10214_c0","rxn09128_c0","rxn10215_c0","rxn10337_c0","rxn05310_c0","rxn03164_c0","rxn08619_c0","rxn02056_c0","bio1","EX_cpd00036_e0","EX_cpd00276_e0","EX_cpd00082_e0","EX_cpd00023_e0","EX_cpd00058_e0","EX_cpd00013_e0","EX_cpd00034_e0","EX_cpd15302_c0","EX_cpd00132_e0","EX_cpd00254_e0","EX_cpd00205_e0","EX_cpd00009_e0","EX_cpd00048_e0","EX_cpd10516_e0","EX_cpd03847_e0","EX_cpd00149_e0","EX_cpd00099_e0","EX_cpd00027_e0","EX_cpd00268_e0","EX_cpd00007_e0","EX_cpd01080_e0","EX_cpd00130_e0","EX_cpd00039_e0","EX_cpd00011_e0","EX_cpd00241_e0","EX_cpd15605_e0","EX_cpd00793_e0","EX_cpd00030_e0","EX_cpd00264_e0","EX_cpd00111_e0","EX_cpd00038_e0","EX_cpd00028_e0","EX_cpd00063_e0","EX_cpd00322_e0","EX_cpd11606_e0","EX_cpd00115_e0","EX_cpd00218_e0","EX_cpd11493_c0","EX_cpd00166_b"]
              
for rxn_id in list_reactions:
    if rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).bounds = (-1,1)  # Restringir flujo entre -5 y 5              


In [17]:
cobra.io.write_sbml_model(model, "modelo_v13tris.xml")

Correccción general del modleo en base a generalidades teóricas: ejemplo las reacciones de desfosforilación son irreversibles, las reacciones de liberación de CO2 también etc

In [20]:

# Cargar el modelo SBML
modelo_path = "modelo_vquince.xml"  # Asegúrate de que el archivo está en el mismo directorio
model = cobra.io.read_sbml_model(modelo_path)

def get_reaction_equations(reaction_ids):
    equations = {}
    for rid in reaction_ids:
        try:
            reaction = model.reactions.get_by_id(rid)
            equation = reaction.build_reaction_string(use_metabolite_names=True)
            equations[rid] = equation
        except KeyError:
            equations[rid] = "Reacción no encontrada en el modelo."
    return equations

# Lista de identificadores de reacciones a consultar (Ejemplo: primeras 10 reacciones)
reaction_ids_to_check = ["EX_cpd15302_b","EX_cpd00012_b","EX_cpd00214_b","EX_cpd00132_b","EX_cpd00027_b","EX_cpd00028_b","rxn00423_c0","rxn05561_c0","rxn00973_c0","rxn12008_c0","rxn00199_c0","rxn09240_c0","rxn00800_c0","rxn05654_c0","rxn00262_c0","rxn01486_c0","rxn02988_c0","rxn00678_c0","rxn00001_c0","rxn11268_c0","rxn00710_c0","rxn00337_c0","rxn05733_c0","rxn00416_c0","rxn00649_c0","rxn00225_c0","rxn00148_c0","rxn05569_c0","rxn00100_c0","rxn00781_c0","rxn05319_c0","rxn05298_c0","rxn01355_c0","rxn00097_c0","rxn00695_c0","rxn00692_c0","rxn00178_c0","rxn01362_c0","rxn10125_c0","rxn00549_c0","rxn10060_c0","rxn00305_c0","rxn00612_c0","rxn05938_c0","rxn03848_c0","rxn00907_c0","rxn05458_c0","rxn03891_c0","rxn01465_c0","rxn01643_c0","rxn00248_c0","rxn00747_c0","rxn01334_c0","rxn10113_c0","rxn05256_c0","rxn02175_c0","rxn01454_c0","rxn00555_c0","rxn10011_c0","rxn10162_c0","rxn00251_c0","rxn00237_c0","rxn00285_c0","rxn08615_c0","rxn00786_c0","rxn01100_c0","rxn00838_c0","rxn00512_c0","rxn01211_c0","rxn01453_c0","rxn00966_c0","rxn08527_c0","rxn00173_c0","rxn01669_c0","rxn10122_c0","rxn08352_c0","rxn00187_c0","rxn05508_c0","rxn01345_c0","rxn00915_c0","rxn01116_c0","rxn00868_c0","rxn00670_c0","rxn01544_c0","rxn00908_c0","rxn02402_c0","rxn01303_c0","rxn00113_c0","rxn00611_c0","rxn01678_c0","rxn01106_c0","rxn08756_c0","rxn00412_c0","rxn03244_c0","rxn00616_c0","rxn02680_c0","rxn11040_c0","rxn10042_c0","rxn00189_c0","rxn01996_c0","rxn00379_c0","rxn02155_c0","rxn01018_c0","rxn00947_c0","rxn01476_c0","rxn03893_c0","rxn00988_c0","rxn00410_c0","rxn00414_c0","rxn00799_c0","rxn01977_c0","rxn00011_c0","rxn02200_c0","rxn00602_c0","rxn01672_c0","rxn00105_c0","rxn01302_c0","rxn02303_c0","rxn00691_c0","rxn00290_c0","rxn01213_c0","rxn00459_c0","rxn00770_c0","rxn05209_c0","rxn00669_c0","rxn05287_c0","rxn01387_c0","rxn00161_c0","rxn01517_c0","rxn00371_c0","rxn01512_c0","rxn08655_c0","rxn03841_c0","rxn01257_c0","rxn09272_c0","rxn00943_c0","rxn03892_c0","rxn05454_c0","rxn05226_c0","rxn00083_c0","rxn00874_c0","rxn00834_c0","rxn00704_c0","rxn00785_c0","rxn00558_c0","rxn00346_c0","rxn01790_c0","rxn05651_c0","rxn03638_c0","rxn10043_c0","rxn00991_c0","rxn01200_c0","rxn02185_c0","rxn00912_c0","rxn02345_c0","rxn03849_c0","rxn08094_c0","rxn00338_c0","rxn01975_c0","rxn00085_c0","rxn00119_c0","rxn05468_c0","rxn01466_c0","rxn05740_c0","rxn02288_c0","rxn00256_c0","rxn00138_c0","rxn00347_c0","rxn02167_c0","rxn01485_c0","rxn00239_c0","rxn01791_c0","rxn00224_c0","rxn01513_c0","rxn05605_c0","rxn00674_c0","rxn01603_c0","rxn00686_c0","rxn00157_c0","rxn05289_c0","rxn05467_c0","rxn00117_c0","rxn01520_c0","rxn00330_c0","rxn00560_c0","rxn00293_c0","rxn01361_c0","rxn05235_c0","rxn02304_c0","rxn02186_c0","rxn00916_c0","rxn05148_c0","EX_cpd00067_e0","EX_cpd00106_e0","EX_cpd00036_e0","EX_cpd00012_e0","EX_cpd00276_e0","EX_cpd00001_e0","EX_cpd00023_e0","EX_cpd00214_e0","EX_cpd15302_c0","EX_cpd00132_e0","EX_cpd00048_e0","EX_cpd00027_e0","EX_cpd00007_e0","EX_cpd00130_e0","EX_cpd00011_e0","EX_cpd00028_e0"]
              
              

# Obtener las ecuaciones de las reacciones especificadas
reaction_equations = get_reaction_equations(reaction_ids_to_check)

# Imprimir los resultados
for rid, eq in reaction_equations.items():
    print(f"{rid}: {eq}")


EX_cpd15302_b: glycogenn_1_c0_b <=> 
EX_cpd00012_b: PPi_e0_b <=> 
EX_cpd00214_b: Palmitate_e0_b <=> 
EX_cpd00132_b: L_Asparagine_e0_b <=> 
EX_cpd00027_b: D_Glucose_e0_b <=> 
EX_cpd00028_b: Heme_e0_b <=> 
rxn00423_c0: Acetyl_CoA_c0 + L_Serine_c0 <=> CoA_c0 + O_Acetyl_L_serine_c0
rxn05561_c0: H_plus__e0 + Fumarate_e0 <=> H_plus__c0 + Fumarate_c0
rxn00973_c0: Citrate_c0 --> Isocitrate_c0
rxn12008_c0: Isopentenyldiphosphate_c0 + all_trans_Heptaprenyl_diphosphate_c0 <-- PPi_c0 + H_plus__c0 + Farnesylfarnesylgeraniol_c0
rxn00199_c0: H_plus__c0 + Oxalosuccinate_c0 --> CO2_c0 + 2_Oxoglutarate_c0
rxn09240_c0: H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_plus__c0 + APS_c0
rxn00800_c0: Adenylosuccinate_c0 <=> AMP_c0 + Fumarate_c0
rxn05654_c0: Succinate_e0 + H_plus__e0 <=> Succinate_c0 + H_plus__c0
rxn00262_c0: H2O_c0 + O2_c0 + L_Aspartate_c0 --> NH3_c0 + H2O2_c0 + Oxaloacetate_c0
rxn01486_c0: Isopentenyldiphosphate_c0 + Farnesyldiphosphate_c0 --> PPi_c0 + H_plus__c

In [25]:
import cobra

def analyze_and_fix_model(model_path):
    # Cargar el modelo metabólico
    model = cobra.io.read_sbml_model(model_path)
    
    # Lista de reacciones a modificar
    reactions_to_check =["EX_cpd15302_b","EX_cpd00012_b","EX_cpd00214_b","EX_cpd00132_b","EX_cpd00027_b","EX_cpd00028_b","rxn00423_c0","rxn05561_c0","rxn00973_c0","rxn12008_c0","rxn00199_c0","rxn09240_c0","rxn00800_c0","rxn05654_c0","rxn00262_c0","rxn01486_c0","rxn02988_c0","rxn00678_c0","rxn00001_c0","rxn11268_c0","rxn00710_c0","rxn00337_c0","rxn05733_c0","rxn00416_c0","rxn00649_c0","rxn00225_c0","rxn00148_c0","rxn05569_c0","rxn00100_c0","rxn00781_c0","rxn05319_c0","rxn05298_c0","rxn01355_c0","rxn00097_c0","rxn00695_c0","rxn00692_c0","rxn00178_c0","rxn01362_c0","rxn10125_c0","rxn00549_c0","rxn10060_c0","rxn00305_c0","rxn00612_c0","rxn05938_c0","rxn03848_c0","rxn00907_c0","rxn05458_c0","rxn03891_c0","rxn01465_c0","rxn01643_c0","rxn00248_c0","rxn00747_c0","rxn01334_c0","rxn10113_c0","rxn05256_c0","rxn02175_c0","rxn01454_c0","rxn00555_c0","rxn10011_c0","rxn10162_c0","rxn00251_c0","rxn00237_c0","rxn00285_c0","rxn08615_c0","rxn00786_c0","rxn01100_c0","rxn00838_c0","rxn00512_c0","rxn01211_c0","rxn01453_c0","rxn00966_c0","rxn08527_c0","rxn00173_c0","rxn01669_c0","rxn10122_c0","rxn08352_c0","rxn00187_c0","rxn05508_c0","rxn01345_c0","rxn00915_c0","rxn01116_c0","rxn00868_c0","rxn00670_c0","rxn01544_c0","rxn00908_c0","rxn02402_c0","rxn01303_c0","rxn00113_c0","rxn00611_c0","rxn01678_c0","rxn01106_c0","rxn08756_c0","rxn00412_c0","rxn03244_c0","rxn00616_c0","rxn02680_c0","rxn11040_c0","rxn10042_c0","rxn00189_c0","rxn01996_c0","rxn00379_c0","rxn02155_c0","rxn01018_c0","rxn00947_c0","rxn01476_c0","rxn03893_c0","rxn00988_c0","rxn00410_c0","rxn00414_c0","rxn00799_c0","rxn01977_c0","rxn00011_c0","rxn02200_c0","rxn00602_c0","rxn01672_c0","rxn00105_c0","rxn01302_c0","rxn02303_c0","rxn00691_c0","rxn00290_c0","rxn01213_c0","rxn00459_c0","rxn00770_c0","rxn05209_c0","rxn00669_c0","rxn05287_c0","rxn01387_c0","rxn00161_c0","rxn01517_c0","rxn00371_c0","rxn01512_c0","rxn08655_c0","rxn03841_c0","rxn01257_c0","rxn09272_c0","rxn00943_c0","rxn03892_c0","rxn05454_c0","rxn05226_c0","rxn00083_c0","rxn00874_c0","rxn00834_c0","rxn00704_c0","rxn00785_c0","rxn00558_c0","rxn00346_c0","rxn01790_c0","rxn05651_c0","rxn03638_c0","rxn10043_c0","rxn00991_c0","rxn01200_c0","rxn02185_c0","rxn00912_c0","rxn02345_c0","rxn03849_c0","rxn08094_c0","rxn00338_c0","rxn01975_c0","rxn00085_c0","rxn00119_c0","rxn05468_c0","rxn01466_c0","rxn05740_c0","rxn02288_c0","rxn00256_c0","rxn00138_c0","rxn00347_c0","rxn02167_c0","rxn01485_c0","rxn00239_c0","rxn01791_c0","rxn00224_c0","rxn01513_c0","rxn05605_c0","rxn00674_c0","rxn01603_c0","rxn00686_c0","rxn00157_c0","rxn05289_c0","rxn05467_c0","rxn00117_c0","rxn01520_c0","rxn00330_c0","rxn00560_c0","rxn00293_c0","rxn01361_c0","rxn05235_c0","rxn02304_c0","rxn02186_c0","rxn00916_c0","rxn05148_c0","EX_cpd00067_e0","EX_cpd00106_e0","EX_cpd00036_e0","EX_cpd00012_e0","EX_cpd00276_e0","EX_cpd00001_e0","EX_cpd00023_e0","EX_cpd00214_e0","EX_cpd15302_c0","EX_cpd00132_e0","EX_cpd00048_e0","EX_cpd00027_e0","EX_cpd00007_e0","EX_cpd00130_e0","EX_cpd00011_e0","EX_cpd00028_e0"]
    
    for rxn_id in reactions_to_check:
        if rxn_id in model.reactions:
            reaction = model.reactions.get_by_id(rxn_id)
            
            # 1️⃣ Si es una reacción de intercambio (EX_), hacerla unidireccional y añadir consumo de ATP
            if rxn_id.startswith("EX_"):
                reaction.bounds = (0, 1000)
                atp = next((m for m in model.metabolites if "ATP" in m.name), None)
                adp = next((m for m in model.metabolites if "ADP" in m.name), None)
                phosphate = next((m for m in model.metabolites if "Phosphate" in m.name), None)
                if atp and adp and phosphate:
                    reaction.add_metabolites({atp: -1, adp: 1, phosphate: 1})
                    print("Exchange corrected")
                    
            
            # 2️⃣ Si tiene ATP en reactivos y ADP o AMP en productos, hacerla unidireccional
            reactants = [met.name for met in reaction.reactants]
            products = [met.name for met in reaction.products]
            
            if "ATP" in str(reactants) and "ADP" in str(products) or "AMP" in str(products):
                reaction.bounds = (0, 1000)
                print("ATP corrected")
            
            # 3️⃣ Si libera CO2 en los productos, hacerla unidireccional
            if "CO2" in str(products):
                reaction.bounds = (0, 1000)
                print("CO2 corrected")
            
            # 4️⃣ Si libera protones en los productos, hacerla unidireccional
            if "H+" in str(products) or "H_plus" in str(products):
                print("H corrected")
                reaction.bounds = (0, 1000)
    
    # Ejecutar Análisis de Variabilidad de Flujo (FVA) para verificar ciclos de regeneración de energía
    fva_result = cobra.flux_analysis.flux_variability_analysis(model)
    high_variability_reactions = fva_result[fva_result["maximum"] > 100].index.tolist()
    
    # 5️⃣ Restringir flujo de reacciones con alta variabilidad
    for rxn_id in high_variability_reactions:
        if rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).bounds = (-5, 5)
            print("Variability corrected")
    
    # Guardar el modelo corregido
    cobra.io.write_sbml_model(model, "modelo_v16.xml")
    print("✅ Modelo corregido guardado como 'modelo_corregido.xml'")
    
# Ejecutar la función con el modelo cargado
analyze_and_fix_model("modelo_vquince.xml")


Exchange corrected
ATP corrected
Exchange corrected
ATP corrected
Exchange corrected
ATP corrected
Exchange corrected
ATP corrected
Exchange corrected
ATP corrected
Exchange corrected
ATP corrected
H corrected
H corrected
CO2 corrected
H corrected
ATP corrected
H corrected
H corrected
H corrected
H corrected
CO2 corrected
ATP corrected
H corrected
ATP corrected
H corrected
ATP corrected
ATP corrected
H corrected
ATP corrected
H corrected
H corrected
ATP corrected
H corrected
ATP corrected
ATP corrected
H corrected
H corrected
H corrected
CO2 corrected
H corrected
CO2 corrected
H corrected
H corrected
H corrected
H corrected
ATP corrected
H corrected
H corrected
H corrected
CO2 corrected
ATP corrected
ATP corrected
ATP corrected
H corrected
H corrected
H corrected
H corrected
H corrected
ATP corrected
H corrected
H corrected
H corrected
H corrected
CO2 corrected
CO2 corrected
ATP corrected
H corrected
H corrected
ATP corrected
ATP corrected
H corrected
H corrected
CO2 corrected
H correc

Asegurar que se han corregido la reacciones

In [43]:
import cobra

# Cargar el modelo SBML
modelo_path = "modelo_v16.xml"  # Asegúrate de que el archivo está en el mismo directorio
model = cobra.io.read_sbml_model(modelo_path)

def get_reaction_equations(reaction_ids):
    equations = {}
    for rid in reaction_ids:
        try:
            reaction = model.reactions.get_by_id(rid)
            equation = reaction.build_reaction_string(use_metabolite_names=True)
            equations[rid] = equation
        except KeyError:
            equations[rid] = "Reacción no encontrada en el modelo."
    return equations

# Lista de identificadores de reacciones a consultar (Ejemplo: primeras 10 reacciones)
reaction_ids_to_check=["EX_cpd00276_b","EX_cpd00023_b","EX_cpd00971_b","EX_cpd00048_b","EX_cpd00011_b","rxn02201_c0","rxn03408_c0","rxn00973_c0","rxn03239_c0","rxn05457_c0","rxn09240_c0","rxn00258_c0","rxn00179_c0","rxn02504_c0","rxn04996_c0","rxn00262_c0","rxn10126_c0","rxn00678_c0","rxn00001_c0","rxn00910_c0","rxn00225_c0","rxn01102_c0","rxn05569_c0","rxn00100_c0","rxn05298_c0","rxn00097_c0","rxn00695_c0","rxn00692_c0","rxn00802_c0","rxn03958_c0","rxn00178_c0","rxn10125_c0","rxn00549_c0","rxn00612_c0","rxn01111_c0","rxn10222_c0","rxn00300_c0","rxn02380_c0","rxn03419_c0","rxn00907_c0","rxn00336_c0","rxn01643_c0","rxn00747_c0","rxn10113_c0","rxn02175_c0","rxn01454_c0","rxn00555_c0","rxn00849_c0","rxn00251_c0","rxn00790_c0","rxn03910_c0","rxn00237_c0","rxn03421_c0","rxn00285_c0","rxn08615_c0","rxn01015_c0","rxn01281_c0","rxn00786_c0","rxn03908_c0","rxn01100_c0","rxn00512_c0","rxn10114_c0","rxn00173_c0","rxn01669_c0","rxn10122_c0","rxn00187_c0","rxn02503_c0","rxn01653_c0","rxn01116_c0","rxn00018_c0","rxn00113_c0","rxn00611_c0","rxn01678_c0","rxn00394_c0","rxn01106_c0","rxn00412_c0","rxn03005_c0","rxn00616_c0","rxn11040_c0","rxn10042_c0","rxn02895_c0","rxn00189_c0","rxn01996_c0","rxn00420_c0","rxn02804_c0","rxn00548_c0","rxn00283_c0","rxn00379_c0","rxn02155_c0","rxn02475_c0","rxn01476_c0","rxn03146_c0","rxn00988_c0","rxn00410_c0","rxn01333_c0","rxn00407_c0","rxn00333_c0","rxn00799_c0","rxn01977_c0","rxn02200_c0","rxn00602_c0","rxn00105_c0","rxn01302_c0","rxn00101_c0","rxn02285_c0","rxn00290_c0","rxn03907_c0","rxn01213_c0","rxn00459_c0","rxn00770_c0","rxn05209_c0","rxn00461_c0","rxn00669_c0","rxn01652_c0","rxn01115_c0","rxn01517_c0","rxn00371_c0","rxn04113_c0","rxn09272_c0","rxn00615_c0","rxn00782_c0","rxn00785_c0","rxn00693_c0","rxn03909_c0","rxn03084_c0","rxn05323_c0","rxn05651_c0","rxn00991_c0","rxn01200_c0","rxn00763_c0","rxn01434_c0","rxn08094_c0","rxn01975_c0","rxn02373_c0","rxn00085_c0","rxn09345_c0","rxn01466_c0","rxn05740_c0","rxn03904_c0","rxn03901_c0","rxn00193_c0","rxn00256_c0","rxn00138_c0","rxn06729_c0","rxn00347_c0","rxn00302_c0","rxn10261_c0","rxn01519_c0","rxn01014_c0","rxn02474_c0","rxn00604_c0","rxn00674_c0","rxn01019_c0","rxn02914_c0","rxn00157_c0","rxn01286_c0","rxn05467_c0","rxn00117_c0","rxn00330_c0","rxn05039_c0","rxn02011_c0","rxn02008_c0","rxn00851_c0","rxn00467_c0","rxn02286_c0","rxn05335_c0","rxn01101_c0","rxn05293_c0","rxn03164_c0","EX_cpd00276_e0","EX_cpd00023_e0","EX_cpd00971_e0","EX_cpd00048_e0","EX_cpd00011_e0"]          

# Obtener las ecuaciones de las reacciones especificadas
reaction_equations = get_reaction_equations(reaction_ids_to_check)

# Imprimir los resultados
for rid, eq in reaction_equations.items():
    print(f"{rid}: {eq}")

EX_cpd00276_b: GLUM_e0_b <=> 
EX_cpd00023_b: L_Glutamate_e0_b <=> 
EX_cpd00971_b: Na_plus__e0_b <=> 
EX_cpd00048_b: Sulfate_e0_b <=> 
EX_cpd00011_b: CO2_e0_b <=> 
rxn02201_c0: ABEE_c0 + 2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydropteridinediphosphate_c0 <=> PPi_c0 + H_plus__c0 + Dihydropteroate_c0
rxn03408_c0: UDP_N_acetylglucosamine_c0 + Undecaprenyl_diphospho_N_acetylmuramoyl_L_alanyl_D_glutamyl_meso_2_6_diaminopimeloyl_D_alanyl_D_alanine_c0 <=> UDP_c0 + Undecaprenyl_diphospho_N_acetylmuramoyl__N_acetylglucosamine_L_ala_D_glu_meso_2_6_diaminopimeloyl_D_ala_D_ala_c0
rxn00973_c0: Citrate_c0 <=> Isocitrate_c0
rxn03239_c0: NAD_c0 + S_3_Hydroxyhexadecanoyl_CoA_c0 <=> NADH_c0 + H_plus__c0 + 3_Oxopalmitoyl_CoA_c0
rxn05457_c0: CoA_c0 + Myristoyl_ACP_c0 <=> Myristoyl_CoA_c0 + ACP_c0
rxn09240_c0: H2O_c0 + ATP_c0 + GTP_c0 + Sulfate_c0 <=> Phosphate_c0 + PPi_c0 + GDP_c0 + H_plus__c0 + APS_c0
rxn00258_c0: Pyruvate_c0 + Malonyl_CoA_c0 <=> Acetyl_CoA_c0 + Oxaloacetate_c0
rxn00179_c0: ATP_c0 + L_Gl

### Simulation on experimental data 

Los unicos datos de crecimiento de dechloromonas que encontré fueron de un paper en los que se crecía la bacteria en medio mínimo anaeróbico, intentamos replicar este experimento aquí para ver si los datos de crecimiento se parecen. La constante de crecimiento deberia estar entre: 0.18–0.26  pero como se ve la simulada es un poco menor. 

Modificamos el medio para que sea similar a las condiciones experimentales

In [40]:
model = cobra.io.read_sbml_model("modelo_v13.xml")

In [41]:
for reaction in model.exchanges: 
    reaction.upper_bound=0.01
    reaction.lower_bound=-0.01

In [42]:
# Lista de reacciones a activar en el medio BG-11
bg11_reacciones = [
    "EX_cpd00001_b",  # H2O
    "EX_cpd00254_b",  # Fosfato
    "EX_cpd00048_b",  # Sulfato
    "EX_cpd00241_b",  # Ca2+
    "EX_cpd00149_b",  # HCO3-
    "EX_cpd10516_b",  # Hierro (citrato férrico amónico)
    "EX_cpd00058_b",  # EDTA / SO4 / Cu2+
    "EX_cpd00137_b",  # Boro
    "EX_cpd00030_b",  # Mn2+
    "EX_cpd00220_b",  # Zn2+
    "EX_cpd00205_b",  # Co2+
    # Opcionalmente:
    "EX_cpd00013_b",  # NH4+ (si reemplazas NO3-)
]

for rxn_id in bg11_reacciones:
    reaction = model.reactions.get_by_id(rxn_id)
    reaction.lower_bound = -100
    reaction.upper_bound = 100


In [43]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00012_b,EX_cpd00012_b,0.005186,0,0.00%
cpd00013_b,EX_cpd00013_b,0.003535,0,0.00%
cpd00023_b,EX_cpd00023_b,0.006472,5,26.49%
cpd00028_b,EX_cpd00028_b,1.76E-05,34,0.49%
cpd00030_b,EX_cpd00030_b,8.801E-06,0,0.00%
cpd00034_b,EX_cpd00034_b,8.801E-06,0,0.00%
cpd00045_b,EX_cpd00045_b,0.0007508,10,6.15%
cpd00048_b,EX_cpd00048_b,0.005839,0,0.00%
cpd00058_b,EX_cpd00058_b,8.801E-06,0,0.00%
cpd00063_b,EX_cpd00063_b,8.801E-06,0,0.00%


In [74]:
import cobra

def update_medium_and_run_fba(model, allowed_metabolites):
    """
    Actualiza el medio del modelo estableciendo a 100 las reacciones EX_ permitidas,
    bloqueando las no permitidas (flujo = 0) y creando las que no existen.
    Luego, ejecuta FBA.
    """
    # Guardar el medio original
    old_medium = model.medium.copy()
    
    # Crear diccionario del nuevo medio
    new_medium = {}
    
    for met in allowed_metabolites:
        rxn_id = f"EX_{met}_e0"
        if rxn_id in model.reactions:
            new_medium[rxn_id] = 100
        else:
            # Crear nueva reacción de intercambio si no existe
            exchange_reaction = cobra.Reaction(rxn_id)
            exchange_reaction.name = f"Exchange for {met}"
            exchange_reaction.lower_bound = -100  # Entrada permitida
            exchange_reaction.upper_bound = 100   # Salida permitida
            
            # Agregar metabolito al modelo si no existe
            if met not in model.metabolites:
                new_metabolite = cobra.Metabolite(met, compartment='e0')
                model.add_metabolites([new_metabolite])
            
            # Asociar metabolito a la reacción
            exchange_reaction.add_metabolites({model.metabolites.get_by_id(met): -1})
            
            # Agregar la reacción al modelo
            model.add_reactions([exchange_reaction])
            
            # Agregar al medio
            new_medium[rxn_id] = 100
    
    # Añadir metabolitos con _b y excluir azúcares con _e0
    additional_metabolites = [ # nitrate
        "cpd00009_b",  # phosphate
        "cpd00048_b",  # sulfate
        "cpd00011_b",  # calcium
        "cpd00137_b",  # citric acid
        "cpd00971_b",  # ferric citrate, # EDTA
        "cpd00149_b",  # carbonate
        "cpd00030_b",  # manganese
        "cpd00254_b",  # molybdate
        "cpd00058_b",  # copper
        "cpd00149_b",  # cobalt (usa el mismo que carbonate)
        ]
    
    for met in additional_metabolites:
        rxn_id = f"EX_{met}"
        if rxn_id not in new_medium:
            new_medium[rxn_id] = 100
    
    # Bloquear todas las demás reacciones de intercambio
    for rxn in model.reactions:
        if rxn.id.startswith("EX_") and rxn.id not in new_medium:
            model.reactions.get_by_id(rxn.id).bounds = (0, 0)
    
    # Aplicar el nuevo medio
    model.medium = new_medium
    
    # Ejecutar FBA
    solution = model.optimize()
    
    return old_medium, new_medium, solution.objective_value

# Cargar el modelo (reemplaza con tu modelo SBML)
model = cobra.io.read_sbml_model("modelo_v13.xml")

# Lista de metabolitos permitidos en el medio
allowed_metabolites = [
    "cpd00001", "cpd00009", "cpd00023", "cpd00027", "cpd00030", "cpd00034", "cpd00035", "cpd00039", "cpd00041", 
    "cpd00053", "cpd00054", "cpd00058", "cpd00060", "cpd00063", "cpd00065", "cpd00066", "cpd00067", "cpd00069", 
    "cpd00084", "cpd00099", "cpd00104", "cpd00107", "cpd00119", "cpd00129", "cpd00132", "cpd00137", "cpd00149", 
    "cpd00156", "cpd00161", "cpd00205", "cpd00215", "cpd00218", "cpd00220", "cpd00254", "cpd00305", "cpd00393", 
    "cpd00644", "cpd10515", "cpd10516", "cpd11574"
]

# Actualizar medio y ejecutar FBA
old_medium, new_medium, growth_rate = update_medium_and_run_fba(model, allowed_metabolites)

# Imprimir resultados
print("Nuevo medio aplicado:")
print(new_medium)
print(f"Tasa de crecimiento después del FBA: {growth_rate}")


Nuevo medio aplicado:
{'EX_cpd00001_e0': 100, 'EX_cpd00009_e0': 100, 'EX_cpd00023_e0': 100, 'EX_cpd00027_e0': 100, 'EX_cpd00030_e0': 100, 'EX_cpd00034_e0': 100, 'EX_cpd00035_e0': 100, 'EX_cpd00039_e0': 100, 'EX_cpd00041_e0': 100, 'EX_cpd00053_e0': 100, 'EX_cpd00054_e0': 100, 'EX_cpd00058_e0': 100, 'EX_cpd00060_e0': 100, 'EX_cpd00063_e0': 100, 'EX_cpd00065_e0': 100, 'EX_cpd00066_e0': 100, 'EX_cpd00067_e0': 100, 'EX_cpd00069_e0': 100, 'EX_cpd00084_e0': 100, 'EX_cpd00099_e0': 100, 'EX_cpd00104_e0': 100, 'EX_cpd00107_e0': 100, 'EX_cpd00119_e0': 100, 'EX_cpd00129_e0': 100, 'EX_cpd00132_e0': 100, 'EX_cpd00137_e0': 100, 'EX_cpd00149_e0': 100, 'EX_cpd00156_e0': 100, 'EX_cpd00161_e0': 100, 'EX_cpd00205_e0': 100, 'EX_cpd00215_e0': 100, 'EX_cpd00218_e0': 100, 'EX_cpd00220_e0': 100, 'EX_cpd00254_e0': 100, 'EX_cpd00305_e0': 100, 'EX_cpd00393_e0': 100, 'EX_cpd00644_e0': 100, 'EX_cpd10515_e0': 100, 'EX_cpd10516_e0': 100, 'EX_cpd11574_e0': 100, 'EX_cpd00009_b': 100, 'EX_cpd00048_b': 100, 'EX_cpd00011_

In [ ]:
import cobra

def compare_media(old_medium, new_medium, model):
    """
    Compara el medio antiguo con el nuevo y muestra qué compuestos están presentes
    en el nuevo medio y cuáles han sido eliminados.
    
    Parámetros:
    - old_medium: Diccionario del medio antiguo.
    - new_medium: Diccionario del medio nuevo.
    - model: Modelo metabólico de COBRA.
    
    Retorna:
    - missing_metabolites: Lista de metabolitos eliminados con su nombre.
    - added_metabolites: Lista de metabolitos añadidos con su nombre.
    """
    old_metabolites = set(old_medium.keys())
    new_metabolites = set(new_medium.keys())
    
    missing_metabolites = old_metabolites - new_metabolites
    added_metabolites = new_metabolites - old_metabolites
    
    missing_info = [(rxn, model.reactions.get_by_id(rxn).name) for rxn in missing_metabolites if rxn in model.reactions]
    added_info = [(rxn, model.reactions.get_by_id(rxn).name) for rxn in added_metabolites if rxn in model.reactions]
    
    return missing_info, added_info

# Ejecutar comparación
missing_metabolites, added_metabolites = compare_media(old_medium, new_medium, model)

# Imprimir resultados
print("Metabolitos eliminados del medio:")
for rxn_id, met_name in missing_metabolites:
    print(f"{rxn_id}: {met_name}")

print("\nMetabolitos añadidos al medio:")
for rxn_id, met_name in added_metabolites:
    print(f"{rxn_id}: {met_name}")


In [79]:
import cobra

def add_metabolites_to_medium(model, additional_metabolites):
    """
    Añade los metabolitos especificados al medio del modelo con un flujo de 100.
    
    Parámetros:
    - model: Modelo metabólico de COBRA.
    - additional_metabolites: Lista de identificadores de metabolitos a añadir.
    
    Retorna:
    - updated_medium: Diccionario con el medio actualizado.
    """
    # Obtener el medio actual
    updated_medium = model.medium.copy()
    
    for met in additional_metabolites:
        rxn_id = f"EX_{met}"
        if rxn_id in model.reactions:
            updated_medium[rxn_id] = 100  # Asignar flujo de entrada de 100
        else:
            # Crear nueva reacción de intercambio si no existe
            exchange_reaction = cobra.Reaction(rxn_id)
            exchange_reaction.name = f"Exchange for {met}"
            exchange_reaction.lower_bound = -100  # Entrada permitida
            exchange_reaction.upper_bound = 100   # Salida permitida
            
            # Agregar metabolito al modelo si no existe
            if met not in model.metabolites:
                new_metabolite = cobra.Metabolite(met, compartment='e0')
                model.add_metabolites([new_metabolite])
            
            # Asociar metabolito a la reacción
            exchange_reaction.add_metabolites({model.metabolites.get_by_id(met): -1})
            
            # Agregar la reacción al modelo
            model.add_reactions([exchange_reaction])
            
            # Agregar al medio
            updated_medium[rxn_id] = 100
    
    # Aplicar el medio actualizado
    model.medium = updated_medium
    
    return updated_medium



# Lista de metabolitos a añadir
additional_metabolites = [
    "cpd00038_e0", "cpd00038_b", "cpd00138_b", "cpd00034_b", "cpd00012_b", "cpd00028_e0", "cpd00028_b", 
    "cpd00588_b", "cpd00205_b", "cpd02701_b", "cpd00039_b", "cpd00531_e0", "cpd00531_b", "cpd00793_b", 
    "cpd00013_e0", "cpd00013_b", "cpd00241_b", "cpd00276_e0", "cpd00276_b", "cpd00122_b", "cpd00063_b", 
    "cpd00322_e0", "cpd00322_b", "cpd03696_b", "cpd00214_b", "cpd00048_b", "cpd00797_b", "cpd15302_b", 
    "cpd01080_b", "cpd00132_b", "cpd00041_b", "cpd00076_b", "cpd15606_e0", "cpd15606_b", "cpd00130_e0", 
    "cpd00130_b", "cpd00099_b", "cpd00106_e0", "cpd00106_b", "cpd00011_b", "cpd01012_b", "cpd00023_b", 
    "cpd01171_b", "cpd00007_b", "cpd00254_b", "cpd00314_b", "cpd00264_b", "cpd01012_e0", "cpd01012_b", 
    "cpd00009_b", "cpd00001_b", "cpd11595_e0", "cpd11595_b", "cpd03847_b", "cpd00268_e0", "cpd00268_b", 
    "cpd15605_b", "cpd00971_b", "cpd10516_b", "cpd01080_e0", "cpd01080_b", "cpd00264_e0", "cpd00264_b", 
    "cpd01171_b", "cpd00115_e0", "cpd00115_b", "cpd15606_b", "cpd00244_e0", "cpd00244_b", "cpd00058_b", 
    "cpd00027_b", "cpd04097_e0", "cpd04097_b", "cpd00218_b", "cpd00214_e0", "cpd00214_b", "cpd15302_c0", 
    "cpd15302_b", "cpd00794_b", "cpd00322_b", "cpd03696_e0", "cpd03696_b", "cpd11416_b", "cpd00241_e0", 
    "cpd00241_b", "cpd00067_b", "cpd00793_e0", "cpd00793_b", "cpd00013_b", "cpd00179_b", "cpd01030_e0", 
    "cpd01030_b", "cpd00971_e0", "cpd00971_b", "cpd00138_b", "cpd00130_b", "cpd00122_e0", "cpd00122_b", 
    "cpd00111_e0", "cpd00111_b", "cpd00106_b", "cpd00012_e0", "cpd00012_b", "cpd00082_b", "cpd00588_b", 
    "cpd11416_c0", "cpd11416_b", "cpd02701_c0", "cpd02701_b", "cpd00115_b", "cpd03847_e0", "cpd03847_b", 
    "cpd04097_b", "cpd00030_b", "cpd01030_b", "cpd00111_b", "cpd00048_e0", "cpd00048_b", "cpd11595_b", 
    "cpd00038_b", "cpd00797_e0", "cpd00797_b", "cpd00036_b", "cpd00149_b", "cpd00794_e0", "cpd00794_b", 
    "cpd15605_e0", "cpd15605_b", "cpd00268_b", "cpd00159_b", "cpd00314_b", "cpd00082_b", "cpd00028_b", 
    "cpd00244_b", "cpd11606_e0", "cpd11606_b", "cpd00011_b", "cpd00011_e0", "cpd00076_e0", "cpd01171_e0", "cpd01171_b", "cpd00159_e0", "cpd00159_b"
]

# Añadir metabolitos al medio
updated_medium = add_metabolites_to_medium(model, additional_metabolites)

# Imprimir resultados
print("Medio actualizado con nuevos metabolitos:")
print(updated_medium)


Medio actualizado con nuevos metabolitos:
{'EX_cpd15302_b': 100, 'EX_cpd02701_b': 100, 'EX_cpd11416_b': 100, 'EX_cpd00067_b': 100, 'EX_cpd00106_b': 100, 'EX_cpd00036_b': 100, 'EX_cpd04097_b': 100, 'EX_cpd00012_b': 100, 'EX_cpd01030_b': 100, 'EX_cpd01012_b': 100, 'EX_cpd00276_b': 100, 'EX_cpd00082_b': 100, 'EX_cpd00001_b': 100, 'EX_cpd00023_b': 100, 'EX_cpd00971_b': 100, 'EX_cpd00058_b': 100, 'EX_cpd00314_b': 100, 'EX_cpd11595_b': 100, 'EX_cpd00797_b': 100, 'EX_cpd00013_b': 100, 'EX_cpd00244_b': 100, 'EX_cpd00179_b': 100, 'EX_cpd00034_b': 100, 'EX_cpd00214_b': 100, 'EX_cpd00076_b': 100, 'EX_cpd01171_b': 100, 'EX_cpd00041_b': 100, 'EX_cpd00122_b': 100, 'EX_cpd03696_b': 100, 'EX_cpd00132_b': 100, 'EX_cpd00588_b': 100, 'EX_cpd00254_b': 100, 'EX_cpd00205_b': 100, 'EX_cpd00138_b': 100, 'EX_cpd00159_b': 100, 'EX_cpd00009_b': 100, 'EX_cpd00048_b': 100, 'EX_cpd10516_b': 100, 'EX_cpd03847_b': 100, 'EX_cpd00149_b': 100, 'EX_cpd00099_b': 100, 'EX_cpd00531_b': 100, 'EX_cpd00027_b': 100, 'EX_cpd0079

In [80]:
import cobra

def run_fba(model):
    """
    Ejecuta un análisis de balance de flujo (FBA) en el modelo metabólico.
    
    Parámetros:
    - model: Modelo metabólico de COBRA.
    
    Retorna:
    - solution: Resultado de la optimización FBA.
    """
    solution = model.optimize()
    
    return solution


# Ejecutar FBA
solution = run_fba(model)

# Imprimir resultados
print("Resultados del FBA:")
print(f"Estado de la optimización: {solution.status}")
print(f"Tasa de crecimiento objetivo: {solution.objective_value}")


Resultados del FBA:
Estado de la optimización: optimal
Tasa de crecimiento objetivo: 0.10526325280638117


In [81]:
import cobra

def print_medium_compounds(model):
    """
    Imprime los compuestos presentes en el medio del modelo FBA con sus nombres y valores asignados.
    
    Parámetros:
    - model: Modelo metabólico de COBRA.
    """
    medium = model.medium
    
    print("Compuestos en el medio del modelo:")
    for rxn_id, value in medium.items():
        if rxn_id in model.reactions:
            reaction = model.reactions.get_by_id(rxn_id)
            metabolite_names = [met.name for met in reaction.metabolites]
            print(f"{rxn_id}: {value}, Metabolitos: {', '.join(metabolite_names)}")



# Imprimir los compuestos del medio del modelo
print_medium_compounds(model)


Compuestos en el medio del modelo:
EX_cpd15302_b: 100, Metabolitos: glycogenn_1_c0_b
EX_cpd02701_b: 100, Metabolitos: S_Adenosyl_4_methylthio_2_oxobutanoate_c0_b
EX_cpd11416_b: 100, Metabolitos: Biomass_c0_b
EX_cpd00067_b: 100, Metabolitos: H_plus__e0_b
EX_cpd00106_b: 100, Metabolitos: Fumarate_e0_b
EX_cpd00036_b: 100, Metabolitos: Succinate_e0_b
EX_cpd04097_b: 100, Metabolitos: Pb_e0_b
EX_cpd00012_b: 100, Metabolitos: PPi_e0_b
EX_cpd01030_b: 100, Metabolitos: Salicin_e0_b
EX_cpd01012_b: 100, Metabolitos: Cd2_plus__e0_b
EX_cpd00276_b: 100, Metabolitos: GLUM_e0_b
EX_cpd00082_b: 100, Metabolitos: D_Fructose_e0_b
EX_cpd00001_b: 100, Metabolitos: H2O_e0_b
EX_cpd00023_b: 100, Metabolitos: L_Glutamate_e0_b
EX_cpd00971_b: 100, Metabolitos: Na_plus__e0_b
EX_cpd00058_b: 100, Metabolitos: Cu2_plus__e0_b
EX_cpd00314_b: 100, Metabolitos: D_Mannitol_e0_b
EX_cpd11595_b: 100, Metabolitos: chromate_e0_b
EX_cpd00797_b: 100, Metabolitos: R_3_Hydroxybutanoate_e0_b
EX_cpd00013_b: 100, Metabolitos: NH3_e0_

In [82]:
import cobra

def restrict_glucose_exchange(model, glucose_ids, lower_bound=0, upper_bound=0):
    """
    Restringe el intercambio de glucosa en el modelo estableciendo sus límites de flujo.
    
    Parámetros:
    - model: Modelo metabólico de COBRA.
    - glucose_ids: Lista de identificadores de glucosa a restringir.
    - lower_bound: Límite inferior del intercambio (por defecto 0, bloqueado).
    - upper_bound: Límite superior del intercambio (por defecto 0, bloqueado).
    
    Retorna:
    - restricted_reactions: Lista de reacciones de intercambio de glucosa restringidas.
    """
    restricted_reactions = []
    
    for glucose_id in glucose_ids:
        rxn_id = f"EX_{glucose_id}"
        if rxn_id in model.reactions:
            reaction = model.reactions.get_by_id(rxn_id)
            reaction.bounds = (lower_bound, upper_bound)
            restricted_reactions.append((rxn_id, reaction.bounds))
    
    return restricted_reactions

# Lista de identificadores de glucosa
glucose_ids = [ "cpd00027_c0", "cpd00027_e0"]

# Restringir el intercambio de glucosa
restricted_reactions = restrict_glucose_exchange(model, glucose_ids)

# Imprimir resultados
print("Reacciones de intercambio de glucosa restringidas:")
for rxn_id, bounds in restricted_reactions:
    print(f"{rxn_id}: {bounds}")


Reacciones de intercambio de glucosa restringidas:
EX_cpd00027_e0: (0, 0)


Hacemos el FBA para ver la tasa de crecimiento 

In [83]:
import cobra

def run_fba(model):
    """
    Ejecuta un análisis de balance de flujo (FBA) en el modelo metabólico.
    
    Parámetros:
    - model: Modelo metabólico de COBRA.
    
    Retorna:
    - solution: Resultado de la optimización FBA.
    """
    solution = model.optimize()
    
    return solution


# Ejecutar FBA
solution = run_fba(model)

# Imprimir resultados
print("Resultados del FBA:")
print(f"Estado de la optimización: {solution.status}")
print(f"Tasa de crecimiento objetivo: {solution.objective_value}")


Resultados del FBA:
Estado de la optimización: optimal
Tasa de crecimiento objetivo: 0.10526325280638117


In [84]:
import cobra

def print_medium_compounds(model):
    """
    Imprime los compuestos presentes en el medio del modelo FBA con sus nombres y valores asignados.
    
    Parámetros:
    - model: Modelo metabólico de COBRA.
    """
    medium = model.medium
    
    print("Compuestos en el medio del modelo:")
    for rxn_id, value in medium.items():
        if rxn_id in model.reactions:
            reaction = model.reactions.get_by_id(rxn_id)
            metabolite_names = [met.name for met in reaction.metabolites]
            print(f"{rxn_id}: {value}, Metabolitos: {', '.join(metabolite_names)}")

modelo=cobra.io.read_sbml_model("modelo_v13.xml")

# Imprimir los compuestos del medio del modelo
print_medium_compounds(modelo)

Compuestos en el medio del modelo:
EX_cpd15302_b: 10.0, Metabolitos: glycogenn_1_c0_b
EX_cpd02701_b: 10.0, Metabolitos: S_Adenosyl_4_methylthio_2_oxobutanoate_c0_b
EX_cpd11416_b: 10.0, Metabolitos: Biomass_c0_b
EX_cpd00067_b: 1.0, Metabolitos: H_plus__e0_b
EX_cpd00106_b: 1.0, Metabolitos: Fumarate_e0_b
EX_cpd00036_b: 10.0, Metabolitos: Succinate_e0_b
EX_cpd04097_b: 10.0, Metabolitos: Pb_e0_b
EX_cpd00012_b: 1.0, Metabolitos: PPi_e0_b
EX_cpd01030_b: 10.0, Metabolitos: Salicin_e0_b
EX_cpd01012_b: 10.0, Metabolitos: Cd2_plus__e0_b
EX_cpd00276_b: 10.0, Metabolitos: GLUM_e0_b
EX_cpd00082_b: 10.0, Metabolitos: D_Fructose_e0_b
EX_cpd00001_b: 10.0, Metabolitos: H2O_e0_b
EX_cpd00023_b: 10.0, Metabolitos: L_Glutamate_e0_b
EX_cpd00971_b: 10.0, Metabolitos: Na_plus__e0_b
EX_cpd00058_b: 1.0, Metabolitos: Cu2_plus__e0_b
EX_cpd00314_b: 10.0, Metabolitos: D_Mannitol_e0_b
EX_cpd11595_b: 10.0, Metabolitos: chromate_e0_b
EX_cpd00797_b: 10.0, Metabolitos: R_3_Hydroxybutanoate_e0_b
EX_cpd00013_b: 10.0, Meta

Metabolitos presentes en el medio de model2 pero ausentes en model1:
EX_cpd00138_e0: EX_D_Mannose_e0
EX_cpd00007_e0: EX_O2_e0
EX_cpd00082_e0: EX_D_Fructose_e0
EX_cpd00314_e0: EX_D_Mannitol_e0
EX_cpd00588_e0: EX_Sorbitol_e0
EX_cpd00027_e0: EX_D_Glucose_e0
EX_cpd00179_e0: EX_Maltose_e0

Metabolitos presentes en el medio de model1 pero ausentes en model2:
EX_cpd00066_e0: Exchange for cpd00066
EX_cpd00054_e0: Exchange for cpd00054
EX_cpd00053_e0: Exchange for cpd00053
EX_cpd11574_e0: Exchange for cpd11574
EX_cpd00119_e0: Exchange for cpd00119
EX_cpd00065_e0: Exchange for cpd00065
EX_cpd00215_e0: Exchange for cpd00215
EX_cpd00305_e0: Exchange for cpd00305
EX_cpd10515_e0: Exchange for cpd10515
EX_cpd00156_e0: Exchange for cpd00156
EX_cpd00220_e0: Exchange for cpd00220
EX_cpd00644_e0: Exchange for cpd00644
EX_cpd00060_e0: Exchange for cpd00060
EX_cpd00084_e0: Exchange for cpd00084
EX_cpd00161_e0: Exchange for cpd00161
EX_cpd00107_e0: Exchange for cpd00107
EX_cpd00104_e0: Exchange for cpd00104